<font color="#CC3D3D"><p>
# Making Numeric Features
    
<font color="black"><p>
- [Specification](#Specification)
- [Read Data](#Read-Data)
- [Preprocessing](#Preprocessing)
- [Numeric Feature Making](#Numeric-Feature-Making)
    - [Add Features Round2](#Add-Features-Round2)
- [Merge Numeric Features](#Merge-Numeric-Features)
- [Deployment](#Deployment)

# Specification

- custid : 고객아이디
- goodcd : 상품코드


**수치형 데이터 (연속형)**
- tot_amt : 방문시 현금 흐름
- dis_amt : 할인금액
- net_amt : 실구매액
- sales_time : 구매시간


**수치형 데이터 (이산형)**
- sales_month : 구매월
- sales_day : 구매일
- inst_mon : 할부개월


**범주형 데이터 (명목형)**
- sales_dayofweek : 구매요일
- inst_fee : 무이자할부여부(0:무이자)
- import_flg : 수입품여부(1:수입, 0:국내)

- str_nm : 지점명(4) 
- team_nm : 상품관리팀이름(4) 
- part_nm : 상품관리파트이름(30)
- buyer_nm : 바이어이름(35)
- pc_nm : 상품군이름(77)
- corner_nm : 코너이름(308)
- brd_nm : 브랜드이름(1873)

In [1]:
from lightgbm import LGBMClassifier

C:\Users\choij\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\choij\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\choij\anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


# Read Data

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import math
import os

In [8]:
# 학습, 평가데이터 불러오기
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')
target = pd.read_csv(os.path.abspath("../input")+"/y_train.csv" , encoding = 'cp949')

In [9]:
train[train['pc_nm']=='사이버쇼핑']

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
578987,27710,14,22,목,1833,본점,2300500019900,올젠,홈쇼핑,사이버쇼핑,인터넷백화점,인터넷백화점,점외,0,85000,4250,80750,3,0


In [10]:
train.drop(index = 578987, inplace = True)

In [11]:
# 같이 전처리 하기 위해서 train과 test를 합침

data = pd.concat([train, test],ignore_index= True)

In [12]:
id_data = pd.DataFrame({'custid': data.custid.unique()})

In [13]:
unique_custid = pd.DataFrame(data.custid.unique()).rename(columns = {0 : 'custid'})

# Preprocessing

In [14]:
# 시간데이터 처리

def hour_process(x):
    x = str(x)
    if len(x) == 4:
        return x[:2]
    elif len(x) == 3:
        return x[:1]
    elif len(x) == 2:
        return '0'

data['sales_hour'] = data['sales_time'].apply(hour_process)

def minute_process(x):
    x = str(x)
    if len(x) == 4:
        return x[2:]
    elif len(x) == 3:
        return x[1:]
    elif len(x) == 2:
        return x
    
data['sales_minute'] = data['sales_time'].apply(minute_process)


data['sales_hour'] = data['sales_hour'].astype(int)
data['sales_minute'] = data['sales_minute'].astype(int)

In [15]:
# 날짜데이터 처리

data['sales_month_origin'] = data['sales_month']
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)
data['year'] = data['sales_month_origin'].apply(lambda x: 2018 if x>12 else 2017)

data['year_month_day'] = data['year'].astype(str) + '_' + data['sales_month'].astype(str) + '_' +\
                                data['sales_day'].astype(str) + '_' + data['sales_time'].astype(str)

data['datetime'] = pd.to_datetime(data['year_month_day'], format = '%Y_%m_%d_%H%M')

In [16]:
data['real_amt'] = ( data['tot_amt'] / data['inst_mon'] ).apply(lambda x : math.trunc(x)) 

In [17]:
data['dist_rate'] = (data['dis_amt']/data['tot_amt'])*100

In [18]:
data['환불여부'] = data['tot_amt'].apply(lambda x : 1 if x < 0 else 0 )

***

In [19]:
# corner_nm 에서 겹치는 값들 처리하기

data.loc[data.corner_nm == '모피.피혁', 'corner_nm'] = '모피/피혁'
data.loc[data.corner_nm == '원목(주니어)', 'corner_nm'] = '원목/주니어'
data.loc[data.corner_nm == '우산,장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '우산장갑', 'corner_nm'] = '우산/장갑'
data.loc[data.corner_nm == '트.단품 ', 'corner_nm'] = '트단품'
data.loc[data.corner_nm == 'TV,VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'TV.VTR', 'corner_nm'] = 'TV/VTR'
data.loc[data.corner_nm == 'GBR  지원', 'corner_nm'] = 'GBR지원'
data.loc[data.corner_nm == '페레  지원', 'corner_nm'] = '페레지원'
data.loc[data.corner_nm == '라디오.카세트', 'corner_nm'] = '라디오/카세트'
data.loc[data.corner_nm == '스포츠용퓸', 'corner_nm'] = '스포츠용품'
data.loc[data.corner_nm == '카세트,전화기', 'corner_nm'] = '전화기/카세트'

In [20]:
# pc_nm 에서 겹치는 값들 처리하기

data.loc[data.pc_nm == '침구,수예', 'pc_nm'] = '침구/수예'
data.loc[data.pc_nm == '디자이너부띠크', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '디자이너부틱', 'pc_nm'] = '디자이너부띠끄'
data.loc[data.pc_nm == '니트,단품,모피', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '니트/단품', 'pc_nm'] = '니트/단품/모피'
data.loc[data.pc_nm == '로얄부틱', 'pc_nm'] = '로얄부띠끄'
data.loc[data.pc_nm == '트랜디 케쥬얼', 'pc_nm'] = '트랜디캐쥬얼'

***

In [21]:
data["sales_season"] = data['sales_month'].apply(lambda x : 'SPRING' if (x>=3) and (x<=5)
                                                  else 'SUMMER' if (x>=6) and (x<=8)
                                                  else 'FALL' if (x>=9) and (x<=11)
                                                  else 'WINTER')

In [22]:
data['tot_amt>=0'] = data['tot_amt'].apply(lambda x : x if x>=0 else 0)
data['dis_amt>=0'] = data['dis_amt'].apply(lambda x : x if x>=0 else 0)
data['net_amt>=0'] = data['net_amt'].apply(lambda x : x if x>=0 else 0)

***

In [23]:
# 환불금액열 만들기

data['refund_tot_amt'] = data['tot_amt'].apply(lambda x : -x if x<0 else 0)
data['refund_dis_amt'] = data['dis_amt'].apply(lambda x : -x if x<0 else 0)

In [24]:
# 백화점 영엄시간 열 만들기

data['sales_time_open'] = data['sales_hour'].apply(lambda x : 1 if (x > 10) and (x < 20) else 0)

In [25]:
# 시간을 임의로 나눈 열 생성

def hour_func(x):
    if 9 <= x < 12 :
        return('12시이전')
    elif 12 <= x < 14 :
        return('12~2시')
    elif 14 <= x < 16 :
        return('2~4시')
    elif 16 <= x < 18 :
        return('4~6시')
    else :
        return('6시이후')  
    
data['hour_split'] = data.sales_hour.apply(hour_func)

In [26]:
# 상하반기

data['상하반기'] = data['sales_month'].apply(lambda x : '상반기' if x in [1,2,3,4,5,6] else '하반기')

In [27]:
data['구매건수'] = data['tot_amt>=0'].apply(lambda x: 1 if x > 0 else 0)
data['환불건수'] = data['refund_tot_amt'].apply(lambda x: 1 if x  > 0 else 0)
data['할인건수'] = data['refund_tot_amt'].apply(lambda x: 1 if x  > 0 else 0)

In [28]:
# 주중, 주말

data['week_split'] = data['sales_dayofweek'].apply(lambda x: '주말' if (x == '토' or x == '일') else '주중')

In [29]:
data['diff_time'] = data.groupby('custid')['datetime'].diff().fillna('00:00:00').astype(str)
data['diff_time'] = data['diff_time'].str.split('days').apply(lambda x: 0 if x[0] == '00:00:00' else x[0]).astype(int)

In [30]:
# 백화점 영엄시간 열 만들기

data['sales_time_open'] = data['sales_hour'].apply(lambda x : 1 if (x > 10) and (x < 20) else 0)

In [31]:
# 쇼핑간격

data['day'] = data['year'].astype(str) + '_' + data['sales_month'].astype(str) + '_' + data['sales_day'].astype(str)
data['shopping_gap_open'] = data.query('sales_time_open == 1').groupby(['custid', 'day'])['datetime']\
                                .diff().dt.total_seconds().fillna(0).div(60).astype(int)
data['shopping_gap_open'] = data['shopping_gap_open'].fillna(0)

In [32]:
# 내점 시 구매 간격

data['shopping_gap'] = data.groupby(['custid', 'day'])['datetime'].diff().dt.total_seconds().fillna(0).div(60).astype(int)
data['shopping_gap'] = data['shopping_gap'].fillna(0)

In [33]:
# 로그변환

data['refund_tot_amt_log'] = np.log1p(data['refund_tot_amt'])
data['refund_dis_amt_log'] = np.log1p(data['refund_dis_amt'])

data['tot_amt>=0_log'] = np.log1p(data['tot_amt>=0'])
data['dis_amt>=0_log'] = np.log1p(data['dis_amt>=0'])

In [34]:
data["sales_date"] = np.where(data["sales_month"] >9 , 
                            (data["sales_month"].astype(str)),("0"+data["sales_month"].astype(str)))

In [35]:
data["time"] = pd.to_datetime(data.sales_time, format = "%H%M")

In [36]:
data['total_sec'] = data['sales_time'].apply(lambda x : x//100*60 + x%100 )

In [37]:
data['refund'] = data['tot_amt'].apply(lambda x : abs(x) if x < 0 else 0 )

In [38]:
data['card_friendly'] =  data.query('net_amt >= 50000 & inst_mon > 1')['inst_fee'].apply(lambda x : 1 if x == 0 else 0)

# Numeric Feature Making

In [39]:
tr = data.copy()

In [40]:
features = []

In [41]:
#환불 여부
t = tr.groupby("custid")["refund"].agg([("refund_count","sum")]).reset_index()
t["refund_bool"] = np.where(t["refund_count"]>0,1,0)
features.append(t);display(t)

,custid,refund_count,refund_bool
0,0,0,0
1,2,1850500,1
2,3,695600,1
3,4,0,0
4,5,0,0
...,...,...,...
35962,49988,0,0
35963,49990,0,0
35964,49992,0,0
35965,49993,0,0


In [42]:
def refund_check(x):
    if x<0:
        return x
    else:
        return 0

- 환불 총액, 평균 , 최대

In [43]:
copy_df = tr.copy()
copy_df["tot_amt_refund"] = copy_df.net_amt.apply(refund_check)
t = copy_df.groupby("custid")["tot_amt_refund"].agg([("amt_refund","sum"),
                                                    ("amt_refund_mean","mean"),
                                                    ("amt_refund_max","min")]).reset_index()
features.append(t);display(t)

,custid,amt_refund,amt_refund_mean,amt_refund_max
0,0,0,0.000000,0
1,2,-1807050,-164277.272727,-1416000
2,3,-660820,-22027.333333,-589950
3,4,0,0.000000,0
4,5,0,0.000000,0
...,...,...,...,...
35962,49988,0,0.000000,0
35963,49990,0,0.000000,0
35964,49992,0,0.000000,0
35965,49993,0,0.000000,0


- 최애브랜드 사용수

In [44]:
def g(x):
    y = x.value_counts()
    y = y.iloc[0]
    return y

f = tr.groupby('custid')['brd_nm'].agg([('love_brd_count', g)]).reset_index()
f = f.fillna(0)
features.append(f);display(f)

,custid,love_brd_count
0,0,4
1,2,3
2,3,3
3,4,1
4,5,5
...,...,...
35962,49988,1
35963,49990,1
35964,49992,1
35965,49993,2


- 시간대별 방문횟수

In [45]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t);display(t)

sales_hour,custid,0,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0,0,0,0,0,0,0,2,0,0,0,0,0,4,5,0,0,0
1,2,0,0,0,0,2,1,0,1,1,0,1,1,1,3,0,0,0
2,3,0,0,0,0,0,3,3,2,3,0,1,5,11,2,0,0,0
3,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,8,1,2,0,1,3,5,6,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0
35963,49990,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
35964,49992,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
35965,49993,0,0,0,0,0,0,0,0,0,0,1,2,1,0,0,0,0


- 시간대별 구매금액

In [46]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index().drop([0,1,8,9],axis = 1)
features.append(t);display(t)

sales_hour,custid,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0,0,0,129000,0,0,0,0,0,843000,770000,0,0,0
1,2,-216000,232000,0,445000,1112000,0,211350,146000,434500,1386000,0,0,0
2,3,0,119200,149800,166000,917500,0,-621000,274000,729000,566000,0,0,0
3,4,0,0,0,0,1045000,0,0,0,0,0,0,0,0
4,5,0,1161144,425000,573000,0,93000,283000,599980,927335,991300,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,573000,0,0,0,0,0,59000,0,0,0,0
35963,49990,0,0,0,0,0,213000,0,0,0,0,0,0,0
35964,49992,0,0,0,0,0,0,0,53000,168000,0,0,0,0
35965,49993,0,0,0,0,0,0,21000,52029,70000,0,0,0,0


- 날짜별 방문횟수

In [47]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()

In [48]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f);display(f)

,custid,month_start,month_mid,month_end
0,0,4,1,6
1,2,5,2,4
2,3,8,11,11
3,4,0,0,4
4,5,9,10,13
...,...,...,...,...
35962,49988,4,0,0
35963,49990,0,0,1
35964,49992,1,1,0
35965,49993,0,0,4


- 날짜별 구매금액

In [49]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()

In [50]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f);display(f)

,custid,month_start,month_mid,month_end
0,0,625000,96000,1021000
1,2,1877000,357350,1516500
2,3,-255000,651800,1903700
3,4,0,0,1045000
4,5,1471218,1528212,2054329
...,...,...,...,...
35962,49988,632000,0,0
35963,49990,0,0,213000
35964,49992,53000,168000,0
35965,49993,0,0,143029


- 월간별 날짜횟수

In [51]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t);display(t)

sales_month,custid,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,1,0,2,0,2,3,0,0,3
1,2,2,0,0,0,0,0,0,3,2,2,1,1
2,3,2,0,3,4,2,1,7,4,3,0,0,4
3,4,0,0,0,0,0,0,4,0,0,0,0,0
4,5,0,0,4,0,2,3,6,4,5,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,1,0,0,0,0,0,0,0,0,0,3,0
35963,49990,0,1,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,1,0,0,1,0,0,0,0,0
35965,49993,4,0,0,0,0,0,0,0,0,0,0,0


- 월간 구매금액

In [52]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(t);display(t)

sales_month,custid,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,294000,0,129000,0,630000,331000,0,0,358000
1,2,-216000,0,0,0,0,0,0,404500,1557000,357350,232000,1416000
2,3,143000,0,1295000,-394000,171000,4200,337800,146200,134800,0,0,462500
3,4,0,0,0,0,0,0,1045000,0,0,0,0,0
4,5,0,0,804000,0,269000,584500,1009380,672249,953335,602400,100812,58083
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,59000,0,0,0,0,0,0,0,0,0,573000,0
35963,49990,0,213000,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,168000,0,0,53000,0,0,0,0,0
35965,49993,143029,0,0,0,0,0,0,0,0,0,0,0


***

In [53]:
#쇼핑시간
time_sum = tr.groupby(['sales_date','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
time_sum['shopping_time'] = (time_sum['time']['max'] - time_sum['time']['min']).dt.total_seconds()
time_sum.drop(['sales_date','time'], axis=1, inplace=True,level=0)
time_sum = time_sum.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(time_sum);time_sum

,custid,shopping_time_mean
0,0,1896.000000
1,2,800.000000
2,3,9180.000000
3,4,2820.000000
4,5,6853.333333
...,...,...
35962,49988,600.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,7380.000000


In [54]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f);display(f)

,custid,dis_rate
0,0,10.000000
1,2,7.272942
2,3,3.397571
3,4,2.500000
4,5,6.718750
...,...,...
35962,49988,5.000000
35963,49990,5.000000
35964,49992,5.000000
35965,49993,1.250000


In [55]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f);display(f)

,custid,sales_time
0,0,1757.090909
1,2,1537.090909
2,3,1604.066667
3,4,1434.000000
4,5,1569.437500
...,...,...
35962,49988,1372.500000
35963,49990,1503.000000
35964,49992,1796.500000
35965,49993,1730.750000


In [56]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df);df

part_nm,custid,남성,비남성,남성part
0,0,113000,118000,48.917749
1,2,445000,2312000,16.140733
2,3,218000,136500,61.495063
3,4,0,60000,0.000000
4,5,1923200,99392,95.085910
...,...,...,...,...
35962,49988,0,157000,0.000000
35963,49990,0,213000,0.000000
35964,49992,0,53000,0.000000
35965,49993,0,21000,0.000000


In [57]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df);df

corner_nm,custid,비화장품,화장품,화장품비율
0,0,455000,1056000,69.887492
1,2,1200000,146000,10.846954
2,3,816000,187000,18.644068
3,4,560000,60000,9.677419
4,5,252000,523000,67.483871
...,...,...,...,...
35962,49988,423000,98000,18.809981
35963,49990,0,213000,100.000000
35964,49992,168000,53000,23.981900
35965,49993,20000,0,0.000000


In [58]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f);display(f)

,custid,real_amt
0,0,55151.272727
1,2,187146.909091
2,3,27360.900000
3,4,52550.000000
4,5,82763.125000
...,...,...
35962,49988,63791.500000
35963,49990,35500.000000
35964,49992,110500.000000
35965,49993,35757.250000


In [59]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f);display(f)

,custid,good_count_mean
0,0,1.571429
1,2,1.571429
2,3,1.363636
3,4,1.000000
4,5,1.454545
...,...,...
35962,49988,1.000000
35963,49990,1.000000
35964,49992,1.000000
35965,49993,1.333333


In [60]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df);df

str_nm,custid,무역점,본점,신촌점,천호점
0,0,460000,1282000,0,0
1,2,3161500,0,0,589350
2,3,462500,0,0,1838000
3,4,1045000,0,0,0
4,5,552000,3858459,391300,252000
...,...,...,...,...,...
35962,49988,0,632000,0,0
35963,49990,213000,0,0,0
35964,49992,0,221000,0,0
35965,49993,0,0,143029,0


In [61]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df);df

team_nm,custid,식품팀,의류패션팀,잡화가용팀
0,0,0,231000,1511000
1,2,0,1081500,2669350
2,3,25800,746200,1528500
3,4,0,985000,60000
4,5,343859,1620400,3089500
...,...,...,...,...
35962,49988,0,475000,157000
35963,49990,0,0,213000
35964,49992,0,168000,53000
35965,49993,53029,0,90000


In [62]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df);df

,custid,import_flg_sum
0,0,7
1,2,1
2,3,0
3,4,1
4,5,6
...,...,...
35962,49988,1
35963,49990,1
35964,49992,1
35965,49993,0


In [63]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df);df

part_nm,custid,가정용품,가정용품파트,골프/유니캐쥬얼,공산품,공산품파트,남성의류,남성정장스포츠,로얄부띠끄,로얄부틱,...,여성캐주얼,여성캐쥬얼,영라이브,영어덜트캐쥬얼,영캐릭터,영플라자,잡화,잡화파트,"케주얼,구두,아동",패션잡화
0,0,0,0,118000,0,0,113000,0,0,0,...,0,0,0,0,0,0,0,1282000,0,0
1,2,2312000,0,404500,0,0,445000,0,0,0,...,0,0,0,232000,0,0,146000,0,0,0
2,3,136500,0,0,25800,0,218000,0,0,0,...,0,0,393200,135000,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,425000,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,99392,0,1923200,488000,0,...,552000,0,0,0,0,0,0,523000,580400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,157000,475000,0
35963,49990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,53000,168000,0
35965,49993,0,0,0,21000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70000


In [64]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df);df

corner_nm,custid,DC캐주얼,GBR지원,L/B침구,N/B침구,NB제화,NB핸드백,TOP디자이너,TV/VCR,TV/VTR,...,행사슈즈,행사핸드백,향수,헤어ACC,헤어악세사리,헤어액세사리,홈데코,화장잡화,화장품,훼미닌부틱
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35965,49993,0,0,0,0,0,0,0,0,0,...,0,0,70000,0,0,0,0,0,0,0


In [65]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df);df

pc_nm,custid,(주)현스포츠아쌤,가구,가전,가전/문화,골프,남성잡화,남여구두,내의란제리,농산물,...,트래디셔널,트래디셔널캐쥬얼,트랜디캐쥬얼,트레디셔널,패션슈즈,패션시즌,피혁A,피혁B,핸드백,화장품
0,0,0,0,0,0,0,0,0,0,0,...,118000,0,0,0,0,0,0,0,0,1056000
1,2,0,0,1200000,0,0,0,0,0,0,...,404500,0,0,0,232000,0,0,0,0,146000
2,3,0,0,0,27000,816000,0,0,0,0,...,0,29000,0,0,135000,0,0,0,0,187000
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60000
4,5,0,0,0,0,991200,0,0,0,115895,...,0,0,166000,447000,0,0,0,0,0,523000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98000
35963,49990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,213000
35964,49992,0,0,0,0,0,0,168000,0,0,...,0,0,0,0,0,0,0,0,0,53000
35965,49993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70000


**[총구매액, 구매건수, 평균구매액, 최대구매액, 최소구매액, 구매액표준편차, 구매액변동계수]**

In [66]:
f = data.groupby('custid')['tot_amt'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균총구매액', np.mean),
                                            ('최대총구매액', np.max),
                                            ('최소총구매액', np.min),
                                            ('구매액표준편차', np.std),   
                                            ('구매액변동계수', lambda x : np.std(x)/np.mean(x))]).reset_index().fillna(0)
features.append(f); display(f)

,custid,총구매액,구매건수,평균총구매액,최대총구매액,최소총구매액,구매액표준편차,구매액변동계수
0,0,1742000,11,158363.636364,455000,39000,118540.518581,0.713699
1,2,3750850,11,340986.363636,1416000,-1416000,791625.029320,2.213534
2,3,2300500,30,76683.333333,621000,-621000,191949.716985,2.461075
3,4,1045000,4,261250.000000,560000,49000,250313.636598,0.829772
4,5,5053759,32,157929.968750,682000,20335,153688.376617,0.957817
...,...,...,...,...,...,...,...,...
35962,49988,632000,4,158000.000000,423000,52000,177822.008387,0.974673
35963,49990,213000,1,213000.000000,213000,213000,0.000000,0.000000
35964,49992,221000,2,110500.000000,168000,53000,81317.279836,0.520362
35965,49993,143029,4,35757.250000,70000,20000,23470.074071,0.568435


In [67]:
#평균할부개월수
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); display(f)

,custid,평균할부개월수
0,0,2.8
1,2,3.5
2,3,2.7
3,4,4.5
4,5,1.9
...,...,...
35962,49988,2.2
35963,49990,6.0
35964,49992,1.0
35965,49993,1.0


**[구매상품다양성]** goodcd

In [68]:
n = data.goodcd.nunique()
f = data.groupby('custid')['goodcd'].agg([('구매상품다양성_goodcd', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f); display(f)

,custid,구매상품다양성_goodcd
0,0,0.000672
1,2,0.000672
2,3,0.002113
3,4,0.000384
4,5,0.002113
...,...,...
35962,49988,0.000384
35963,49990,0.000096
35964,49992,0.000192
35965,49993,0.000288


In [69]:
#구매상품다양성['brd_nm']
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f); display(f)

,custid,구매상품다양성
0,0,0.023411
1,2,0.023411
2,3,0.070234
3,4,0.013378
4,5,0.070234
...,...,...
35962,49988,0.013378
35963,49990,0.003344
35964,49992,0.006689
35965,49993,0.010033


In [70]:
#수입상품_구매비율
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f); display(f)

,custid,수입상품_구매비율
0,0,63.6
1,2,9.1
2,3,0.0
3,4,25.0
4,5,18.8
...,...,...
35962,49988,25.0
35963,49990,100.0
35964,49992,50.0
35965,49993,0.0


In [71]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2);test2

,custid,일평균구매액
0,0,3.484000e+05
1,2,6.251417e+05
2,3,2.556111e+05
3,4,1.045000e+06
4,5,5.615288e+05
...,...,...
35962,49988,3.160000e+05
35963,49990,2.130000e+05
35964,49992,1.105000e+05
35965,49993,1.430290e+05


In [72]:
#내점일수 

f = tr.groupby(by = 'custid')['sales_time'].agg([('내점일수','nunique')]).reset_index()
features.append(f); display(f)

,custid,내점일수
0,0,10
1,2,9
2,3,25
3,4,4
4,5,27
...,...,...
35962,49988,4
35963,49990,1
35964,49992,2
35965,49993,4


In [73]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f); display(f)

,custid,일평균구매건
0,0,2.200000
1,2,1.833333
2,3,3.333333
3,4,4.000000
4,5,3.555556
...,...,...
35962,49988,2.000000
35963,49990,1.000000
35964,49992,1.000000
35965,49993,4.000000


In [74]:
#아동상품 구매건수
f = tr[tr.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features.append(f); display(f)

,custid,baby_sales
0,0,0
1,2,1
2,3,1
3,4,0
4,5,10
...,...,...
35962,49988,2
35963,49990,0
35964,49992,1
35965,49993,1


In [75]:
#상위 브랜드 40개의 구매빈도
top_brd = tr["brd_nm"].value_counts()[:40].index
df = tr.groupby(['custid','brd_nm'])["custid"].agg([('brd_count', 'count')]).reset_index()
df =df.pivot_table(values='brd_count', index=df.custid, columns="brd_nm", aggfunc='count',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df);df

,custid,식품,지오다노,랑콤,에스티로더,샤넬,크리니크,폴로,크리스챤디올화장품,아모레,...,마인,타임,클라란스,닥스,아가방,폴로화장품,스포트리플레이,올리비에,월비스타킹,게스
0,0,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35965,49993,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [76]:
#상위 브랜드 40개의 구매금액
df = tr.groupby(['custid','brd_nm'])["tot_amt"].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns="brd_nm", aggfunc='sum',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df);df

,custid,식품,지오다노,랑콤,에스티로더,샤넬,크리니크,폴로,크리스챤디올화장품,아모레,...,마인,타임,클라란스,닥스,아가방,폴로화장품,스포트리플레이,올리비에,월비스타킹,게스
0,0,0,0,394000,90000,0,533000,118000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,146000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,25800,119200,0,0,0,0,0,0,0,...,0,0,0,0,0,187000,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,343859,109400,276000,0,43000,204000,447000,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,98000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,213000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,53000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35965,49993,53029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,70000,0,0,0,0


In [77]:
#주말방문비율
day = {'월' : 0 , '화' : 1 , '수' : 2 , '목' : 3 , '금' : 4 , '토': 5 , '일' : 6 }

tr['sales_dayofweek_num'] = tr['sales_dayofweek'].apply(lambda x : day[x] )

f = tr.groupby('custid')['sales_dayofweek_num'].agg([
    ('주말방문비율', lambda x: np.mean(x >4))]).reset_index()

features.append(f); display(f)

,custid,주말방문비율
0,0,0.636364
1,2,0.363636
2,3,0.366667
3,4,0.250000
4,5,0.156250
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.500000
35965,49993,0.000000


In [78]:
#계절별 구매 건수
def season(k):
    if 3 <= k <= 5 :
        return('봄')
    elif 6 <= k <= 8 :
        return('여름')
    elif 9 <= k <= 11 :    
        return('가을')
    else :
        return('겨울')
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); display(f)

season,custid,가을,겨울,봄,여름
0,0,3,3,1,4
1,2,5,3,0,3
2,3,3,6,9,12
3,4,0,0,0,4
4,5,12,1,6,13
...,...,...,...,...,...
35962,49988,3,1,0,0
35963,49990,0,1,0,0
35964,49992,0,0,1,1
35965,49993,0,4,0,0


In [79]:
#계절별 구매 금액
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc="sum", fill_value=0).reset_index()
features.append(f); display(f)

season,custid,가을,겨울,봄,여름
0,0,331000,358000,294000,759000
1,2,2146350,1200000,0,404500
2,3,134800,605500,1072000,488200
3,4,0,0,0,1045000
4,5,1656547,58083,1073000,2266129
...,...,...,...,...,...
35962,49988,573000,59000,0,0
35963,49990,0,213000,0,0
35964,49992,0,0,168000,53000
35965,49993,0,143029,0,0


In [80]:
#아침점심저녁별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침')
    elif 13 <= x <= 16 :
        return('점심')
    else :
        return('저녁')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc=np.size).reset_index()
f = f.fillna(0)
features.append(f); display(f)

goodmea,custid,아침,저녁,점심
0,0,2.0,9.0,0.0
1,2,3.0,5.0,3.0
2,3,6.0,18.0,6.0
3,4,0.0,0.0,4.0
4,5,9.0,17.0,6.0
...,...,...,...,...
35962,49988,3.0,1.0,0.0
35963,49990,0.0,0.0,1.0
35964,49992,0.0,2.0,0.0
35965,49993,0.0,3.0,1.0


In [81]:
#아침점심저녁별 구매금액
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc="sum").reset_index()
features.append(f); display(f)

goodmea,custid,아침,저녁,점심
0,0,129000.0,1613000.0,NaN
1,2,16000.0,1966500.0,1768350.0
2,3,269000.0,1569000.0,462500.0
3,4,NaN,NaN,1045000.0
4,5,1586144.0,2518615.0,949000.0
...,...,...,...,...
35962,49988,573000.0,59000.0,NaN
35963,49990,NaN,NaN,213000.0
35964,49992,NaN,221000.0,NaN
35965,49993,NaN,122029.0,21000.0


In [82]:
#주방문요일
f = tr.groupby('custid')['sales_dayofweek'].agg([('주방문요일', lambda x: x.value_counts().index[0])]).reset_index()
f = pd.get_dummies(f, columns=['주방문요일'])

features.append(f); display(f)

,custid,주방문요일_금,주방문요일_목,주방문요일_수,주방문요일_월,주방문요일_일,주방문요일_토,주방문요일_화
0,0,0,0,0,0,1,0,0
1,2,0,0,1,0,0,0,0
2,3,0,1,0,0,0,0,0
3,4,0,1,0,0,0,0,0
4,5,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,1
35963,49990,0,0,1,0,0,0,0
35964,49992,0,0,0,0,1,0,0
35965,49993,0,0,1,0,0,0,0


In [83]:
#평균 쇼핑시간
f = tr.groupby('custid')['total_sec'].agg([
    ('평균쇼핑시간', lambda x: (x.max() - x.min()) / x.nunique())]).reset_index()

features.append(f); display(f)

,custid,평균쇼핑시간
0,0,43.800000
1,2,56.666667
2,3,20.400000
3,4,11.750000
4,5,18.222222
...,...,...
35962,49988,85.000000
35963,49990,0.000000
35964,49992,36.500000
35965,49993,30.750000


In [84]:
#일별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f); display(f)

sales_day,custid,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,0,0,3,0,0,0,0,0,1,...,1,0,0,2,2,0,0,1,0,0
1,2,0,0,2,0,1,1,0,1,0,...,1,0,0,0,0,3,0,0,0,0
2,3,4,0,0,0,0,0,3,0,1,...,0,1,0,6,0,3,1,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,1,0
4,5,1,1,0,3,0,0,2,0,1,...,5,1,1,0,0,0,0,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,1,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35965,49993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [85]:
#일별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f); display(f)

sales_day,custid,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,0,0,331000,0,0,0,0,0,294000,...,166000,0,0,129000,630000,0,0,96000,0,0
1,2,0,0,-216000,0,232000,445000,0,1416000,0,...,1112000,0,0,0,0,404500,0,0,0,0
2,3,139000,0,0,0,0,0,227000,0,-621000,...,0,621000,0,1136500,0,119200,27000,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,485000,0,0,560000,0
4,5,24000,58083,0,703000,0,0,240800,0,20335,...,1307000,43000,85000,0,0,0,0,150500,0,223249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,59000,0,0,0,0,573000,0,0,...,0,0,0,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35964,49992,0,53000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35965,49993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,143029


In [86]:
#월별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f); display(f)

sales_month,custid,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,1,0,2,0,2,3,0,0,3
1,2,2,0,0,0,0,0,0,3,2,2,1,1
2,3,2,0,3,4,2,1,7,4,3,0,0,4
3,4,0,0,0,0,0,0,4,0,0,0,0,0
4,5,0,0,4,0,2,3,6,4,5,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,1,0,0,0,0,0,0,0,0,0,3,0
35963,49990,0,1,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,1,0,0,1,0,0,0,0,0
35965,49993,4,0,0,0,0,0,0,0,0,0,0,0


In [87]:
#월별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f); display(f)

sales_month,custid,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,294000,0,129000,0,630000,331000,0,0,358000
1,2,-216000,0,0,0,0,0,0,404500,1557000,357350,232000,1416000
2,3,143000,0,1295000,-394000,171000,4200,337800,146200,134800,0,0,462500
3,4,0,0,0,0,0,0,1045000,0,0,0,0,0
4,5,0,0,804000,0,269000,584500,1009380,672249,953335,602400,100812,58083
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,59000,0,0,0,0,0,0,0,0,0,573000,0
35963,49990,0,213000,0,0,0,0,0,0,0,0,0,0
35964,49992,0,0,0,168000,0,0,53000,0,0,0,0,0
35965,49993,143029,0,0,0,0,0,0,0,0,0,0,0


***

# *1등팀것들 추가*

**[베스트셀러Top-100구매건수,총구매액,평균구매액,구매비율]** 

In [88]:
f = unique_custid
best_seller = list(data.goodcd.value_counts().index[:100])
bstn= data.query('goodcd == @best_seller').groupby('custid').agg({
                                                                '구매건수': [('베스트셀러_구매건수', np.sum)],
                                                                '환불건수': [('베스트셀러_환불건수', np.sum)],
                                                                'tot_amt>=0': [
                                                                            ('베스트셀러_총구매액_합', np.sum),
                                                                            ('베스트셀러_총구매액_평균', np.mean),
                                                                            ('베스트셀러_총구매액_표준편차', np.std),
                                                                         
                                                                            ('베스트셀러_총구매액_최대구매액', np.max),
                                                                ('베스트셀러_총구매액_변동계수',lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))],
                                                                'dis_amt>=0':[
                                                                            ('베스트셀러_총할인액_합', np.sum),
                                                                            ('베스트셀러_총할인액_평균', np.mean),
                                                                            ('베스트셀러_총할인액_표준편차', np.std),
                                                                         
                                                                            ('베스트셀러_총할인액_최대구매액', np.max),
                                                                ('베스트셀러_총할인액_변동계수',lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))],
    
                                                                'inst_mon':[
                                                                            ('베스트셀러_무이자할부총합', np.sum),
                                                                            ('베스트셀러_평균무이자할부개월', np.mean),
                                                                           ('베스트셀러_최소무이자할부개월', np.min),
                                                                           ('베스트셀러_최대무이자할부개월', np.max)]
})

bstn.columns = bstn.columns.droplevel(0)
f = pd.merge(f, bstn, on = 'custid', how = 'left').fillna(0)

features.append(f); display(f)

,custid,베스트셀러_구매건수,베스트셀러_환불건수,베스트셀러_총구매액_합,베스트셀러_총구매액_평균,베스트셀러_총구매액_표준편차,베스트셀러_총구매액_최대구매액,베스트셀러_총구매액_변동계수,베스트셀러_총할인액_합,베스트셀러_총할인액_평균,베스트셀러_총할인액_표준편차,베스트셀러_총할인액_최대구매액,베스트셀러_총할인액_변동계수,베스트셀러_무이자할부총합,베스트셀러_평균무이자할부개월,베스트셀러_최소무이자할부개월,베스트셀러_최대무이자할부개월
0,0,9.0,0.0,1174000.0,130444.444444,73698.899434,294000.0,0.532671,117400.0,13044.444444,7369.889943,29400.0,0.532671,25.0,2.777778,1.0,3.0
1,2,1.0,0.0,146000.0,146000.000000,0.000000,146000.0,0.000000,14600.0,14600.000000,0.000000,14600.0,0.000000,1.0,1.000000,1.0,1.0
2,3,9.0,1.0,719600.0,71960.000000,67939.764171,187000.0,0.895683,33950.0,3395.000000,3481.236211,9350.0,0.972781,22.0,2.200000,1.0,3.0
3,4,1.0,0.0,60000.0,60000.000000,0.000000,60000.0,0.000000,3000.0,3000.000000,0.000000,3000.0,0.000000,2.0,2.000000,2.0,2.0
4,5,16.0,0.0,1577759.0,98609.937500,56891.964798,204000.0,0.558619,123390.0,7711.875000,6954.412718,20400.0,0.873144,28.0,1.750000,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,1.0,0.0,98000.0,98000.000000,0.000000,98000.0,0.000000,4900.0,4900.000000,0.000000,4900.0,0.000000,3.0,3.000000,3.0,3.0
35963,49990,1.0,0.0,213000.0,213000.000000,0.000000,213000.0,0.000000,10650.0,10650.000000,0.000000,10650.0,0.000000,6.0,6.000000,6.0,6.0
35964,49992,1.0,0.0,53000.0,53000.000000,0.000000,53000.0,0.000000,2650.0,2650.000000,0.000000,2650.0,0.000000,1.0,1.000000,1.0,1.0
35965,49993,3.0,0.0,123029.0,41009.666667,25704.849354,70000.0,0.511780,3500.0,1166.666667,2020.725942,3500.0,1.414214,3.0,1.000000,1.0,1.0


**[단독상품구매건수,구매비율]**

In [89]:
f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)])
f2 = data.groupby(['custid','goodcd'])['구매건수'].agg([
            ('구매건수', np.sum)]).reset_index().query('구매건수 == 1').groupby('custid')['구매건수'].agg([
            ('단독상품구매건수', 'count')]).reset_index()

f = pd.merge(f,f2, on = 'custid', how='left').fillna(0)
f['단독상품구매비율'] = f['단독상품구매건수']/f['구매건수']
f = f.iloc[:,[0,2,3]]

features.append(f); display(f)

,custid,단독상품구매건수,단독상품구매비율
0,0,5.0,0.454545
1,2,5.0,0.555556
2,3,18.0,0.642857
3,4,4.0,1.000000
4,5,17.0,0.531250
...,...,...,...
35962,49988,4.0,1.000000
35963,49990,1.0,1.000000
35964,49992,2.0,1.000000
35965,49993,2.0,0.500000


**[환불건수]**

In [90]:
f = data.groupby('custid')['환불건수'].agg('sum').reset_index()
features.append(f); display(f)

,custid,환불건수
0,0,0
1,2,2
2,3,2
3,4,0
4,5,0
...,...,...
35962,49988,0
35963,49990,0
35964,49992,0
35965,49993,0


**[환불금액로그]**

In [91]:
f = data.groupby('custid')['refund_tot_amt_log'].agg([('환불금액log총합', np.sum),
                                              ('환불금액log평균', np.mean),
                                             ('환불금액log표준편차', np.std),
                                            ('환불금액log변동계수',lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))
                                             ]).reset_index().fillna(0)
features.append(f); display(f)

,custid,환불금액log총합,환불금액log평균,환불금액log표준편차,환불금액log변동계수
0,0,0.000000,0.000000,0.000000,0.000000
1,2,27.145301,2.467755,5.496759,2.123774
2,3,24.558997,0.818633,3.127811,3.756553
3,4,0.000000,0.000000,0.000000,0.000000
4,5,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,0.000000


**[fm 스코어]**

In [92]:
f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index() 
f['총구매액'] = data.groupby('custid')['tot_amt>=0'].agg([('총구매액', np.sum)]).reset_index().총구매액

f['f'] = pd.qcut(f['구매건수'], q = 4, labels = range(1, 5)).astype(int)
f['m'] = pd.qcut(f['총구매액'], q = 4, labels = range(1, 5)).astype(int)
f['fm_스코어'] =  (3 * f['f']) + (2 * f['m'])

f = f.iloc[:,[0,5]]
features.append(f); display(f)

,custid,fm_스코어
0,0,12
1,2,14
2,3,15
3,4,7
4,5,17
...,...,...
35962,49988,5
35963,49990,5
35964,49992,5
35965,49993,5


**[고,저가상품 구매액, 구매액비율, 구매건수, 구매건수비율]**

In [93]:
high_low_4q = list(data.tot_amt.describe().reset_index().iloc[4:7,1])

f = data.groupby('custid')['tot_amt>=0'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()

h = data.query('tot_amt > @high_low_4q[2]').groupby('custid')['tot_amt>=0'].agg([('고가상품구매액','sum'),
                                                                                ('고가상품구매건수','count')]).reset_index()
l = data.query('tot_amt < @high_low_4q[0]').groupby('custid')['tot_amt>=0'].agg([('저가상품구매액','sum'),
                                                                                ('저가상품구매건수','count')]).reset_index()
f = pd.merge(f, h, how = 'left').fillna(0)
f = pd.merge(f, l, how = 'left').fillna(0)
f['고가상품구매액비율'] = f['고가상품구매액']/f['총구매액']
f['저가상품구매액비율'] = f['저가상품구매액']/f['총구매액']
f['고가상품구매건수비율'] = f['고가상품구매건수']/f['구매건수']
f['저가상품구매건수비율'] = f['저가상품구매건수']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6,7,8,9,10]]
features.append(f); display(f)

,custid,고가상품구매액,고가상품구매건수,저가상품구매액,저가상품구매건수,고가상품구매액비율,저가상품구매액비율,고가상품구매건수비율,저가상품구매건수비율
0,0,1090000.0,4.0,0.0,0.0,0.625718,0.000000,0.363636,0.000000
1,2,5601350.0,9.0,0.0,2.0,1.000000,0.000000,0.818182,0.181818
2,3,1617000.0,5.0,25800.0,5.0,0.539702,0.008611,0.166667,0.166667
3,4,936000.0,2.0,0.0,0.0,0.895694,0.000000,0.500000,0.000000
4,5,3938049.0,14.0,20335.0,1.0,0.779232,0.004024,0.437500,0.031250
...,...,...,...,...,...,...,...,...,...
35962,49988,423000.0,1.0,0.0,0.0,0.669304,0.000000,0.250000,0.000000
35963,49990,213000.0,1.0,0.0,0.0,1.000000,0.000000,1.000000,0.000000
35964,49992,168000.0,1.0,0.0,0.0,0.760181,0.000000,0.500000,0.000000
35965,49993,0.0,0.0,41000.0,2.0,0.000000,0.286655,0.000000,0.500000


**[총할인금액, 할인건수, 평균할인금액, 최대할인금액, 최소할인금액, 할인금액표준편차, 할인금액변동계수]**

In [94]:
f = data.groupby('custid')['dis_amt>=0'].agg([('총할인금액', np.sum),
                                            # ('할인건수', np.size),
                                            ('평균할인금액', np.mean),
                                            ('최대할인금액', np.max),
                                            ('최소할인금액', np.min),
                                            ('할인금액표준편차', np.std),
                                            ('할인금액변동계수',lambda x : np.std(x) / 0.0001 \
                                             if np.mean(x) == 0 else np.std(x) / np.mean(x))]).reset_index().fillna(0)

features.append(f); display(f)

,custid,총할인금액,평균할인금액,최대할인금액,최소할인금액,할인금액표준편차,할인금액변동계수
0,0,174200,15836.363636,45500,3900,11854.051858,0.713699
1,2,298540,27140.000000,111200,0,33114.527024,1.163355
2,3,126440,4214.666667,31050,0,7216.009373,1.683341
3,4,21800,5450.000000,18800,0,9011.659115,1.431986
4,5,361260,11289.375000,68200,0,14931.151013,1.301755
...,...,...,...,...,...,...,...
35962,49988,31600,7900.000000,21150,2600,8891.100419,0.974673
35963,49990,10650,10650.000000,10650,10650,0.000000,0.000000
35964,49992,11050,5525.000000,8400,2650,4065.863992,0.520362
35965,49993,3500,875.000000,3500,0,1750.000000,1.732051


**[평균할인율]**

In [95]:
amount_mean = data.groupby('custid')['tot_amt>=0'].agg([('평균구매가격', 'mean')])
discount_mean = data.groupby('custid')['dis_amt>=0'].agg([('평균할인금액', 'mean')])
f = pd.DataFrame(discount_mean['평균할인금액'] / amount_mean['평균구매가격']).rename(columns={0: '평균할인율'}).reset_index()
features.append(f); display(f)

,custid,평균할인율
0,0,0.100000
1,2,0.053298
2,3,0.042202
3,4,0.020861
4,5,0.071483
...,...,...
35962,49988,0.050000
35963,49990,0.050000
35964,49992,0.050000
35965,49993,0.024471


**[시간대별 구매건수]**

In [96]:
f = pd.pivot_table(data = data, columns = 'hour_split', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()

dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f);display(f)

hour_split,custid,12~2시_구매건수,12시이전_구매건수,2~4시_구매건수,4~6시_구매건수,6시이후_구매건수
0,0,2,0,0,0,9
1,2,1,2,1,2,3
2,3,5,2,3,5,13
3,4,0,0,4,0,0
4,5,3,8,1,8,12
...,...,...,...,...,...,...
35962,49988,3,0,0,0,1
35963,49990,0,0,1,0,0
35964,49992,0,0,0,1,1
35965,49993,0,0,0,3,1


**[시간대별 환불건수]**

In [97]:
f = pd.pivot_table(data = data, columns = 'hour_split', index = 'custid', values = '환불건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_환불건수'
f.rename(columns = dic, inplace = True)
features.append(f); display(f)

hour_split,custid,12~2시_환불건수,12시이전_환불건수,2~4시_환불건수,4~6시_환불건수,6시이후_환불건수
0,0,0,0,0,0,0
1,2,0,1,0,0,1
2,3,0,1,0,1,0
3,4,0,0,0,0,0
4,5,0,0,0,0,0
...,...,...,...,...,...,...
35962,49988,0,0,0,0,0
35963,49990,0,0,0,0,0
35964,49992,0,0,0,0,0
35965,49993,0,0,0,0,0


**[시간대별 그외다수]** 

In [98]:
cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'hour_split', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        
        dic2 = {}
        g = unique_custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
            
        
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
   
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,12~2시_tot_amt>=0_sum,12시이전_tot_amt>=0_sum,2~4시_tot_amt>=0_sum,4~6시_tot_amt>=0_sum,6시이후_tot_amt>=0_sum
0,0,129000,0,0,0,1613000
1,2,445000,1432000,1112000,357350,2255000
2,3,315800,193800,917500,274000,1295000
3,4,0,0,1045000,0,0
4,5,998000,1161144,93000,882980,1918635
...,...,...,...,...,...,...
35962,49988,573000,0,0,0,59000
35963,49990,0,0,213000,0,0
35964,49992,0,0,0,53000,168000
35965,49993,0,0,0,73029,70000


,custid,12~2시_tot_amt>=0_mean,12시이전_tot_amt>=0_mean,2~4시_tot_amt>=0_mean,4~6시_tot_amt>=0_mean,6시이후_tot_amt>=0_mean
0,0,64500.000000,0.000000,0.000000e+00,0.000000,179222.222222
1,2,445000.000000,477333.333333,1.112000e+06,178675.000000,563750.000000
2,3,63160.000000,64600.000000,3.058333e+05,45666.666667,99615.384615
3,4,0.000000,0.000000,2.612500e+05,0.000000,0.000000
4,5,332666.666667,145143.000000,9.300000e+04,110372.500000,159886.250000
...,...,...,...,...,...,...
35962,49988,191000.000000,0.000000,0.000000e+00,0.000000,59000.000000
35963,49990,0.000000,0.000000,2.130000e+05,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000e+00,53000.000000,168000.000000
35965,49993,0.000000,0.000000,0.000000e+00,24343.000000,70000.000000


,custid,12~2시_tot_amt>=0_max,12시이전_tot_amt>=0_max,2~4시_tot_amt>=0_max,4~6시_tot_amt>=0_max,6시이후_tot_amt>=0_max
0,0,90000,0,0,0,455000
1,2,445000,1200000,1112000,211350,1416000
2,3,98000,119200,621000,89000,539000
3,4,0,0,560000,0,0
4,5,488000,682000,93000,204000,434000
...,...,...,...,...,...,...
35962,49988,423000,0,0,0,59000
35963,49990,0,0,213000,0,0
35964,49992,0,0,0,53000,168000
35965,49993,0,0,0,32029,70000


,custid,12~2시_tot_amt>=0_std,12시이전_tot_amt>=0_std,2~4시_tot_amt>=0_std,4~6시_tot_amt>=0_std,6시이후_tot_amt>=0_std
0,0,36062.445841,0.000000,0.000000,0.000000,121285.590424
1,2,0.000000,636507.135336,0.000000,46209.428151,601722.596662
2,3,35935.052525,60225.908046,275679.312487,30024.434494,138501.731915
3,4,0.000000,0.000000,250313.636598,0.000000,0.000000
4,5,216786.377186,221006.156683,0.000000,72244.175593,114625.099170
...,...,...,...,...,...,...
35962,49988,202230.067003,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,6675.024120,0.000000


,custid,12~2시_tot_amt>=0_변동계수,12시이전_tot_amt>=0_변동계수,2~4시_tot_amt>=0_변동계수,4~6시_tot_amt>=0_변동계수,6시이후_tot_amt>=0_변동계수
0,0,0.559108,0.000000,0.000000,0.000000,0.676733
1,2,0.000000,1.333465,0.000000,0.258623,1.067357
2,3,0.568953,0.932290,0.901404,0.657469,1.390365
3,4,0.000000,0.000000,0.958138,0.000000,0.000000
4,5,0.651662,1.522679,0.000000,0.654549,0.716917
...,...,...,...,...,...,...
35962,49988,1.058796,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,0.274207,0.000000


...

,custid,12~2시_dis_amt>=0_log_sum,12시이전_dis_amt>=0_log_sum,2~4시_dis_amt>=0_log_sum,4~6시_dis_amt>=0_log_sum,6시이후_dis_amt>=0_log_sum
0,0,17.374079,0.000000,0.000000,0.000000,86.759777
1,2,10.703267,10.051951,11.619095,19.547815,21.287236
2,3,25.029903,16.784876,27.420780,31.930075,51.649419
3,4,0.000000,0.000000,17.848366,0.000000,0.000000
4,5,29.807604,44.828123,9.137877,55.994042,66.602060
...,...,...,...,...,...,...
35962,49988,26.320288,0.000000,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,7.882692,9.036106
35965,49993,0.000000,0.000000,0.000000,0.000000,8.160804


,custid,12~2시_dis_amt>=0_log_mean,12시이전_dis_amt>=0_log_mean,2~4시_dis_amt>=0_log_mean,4~6시_dis_amt>=0_log_mean,6시이후_dis_amt>=0_log_mean
0,0,8.687040,0.000000,0.000000,0.000000,9.639975
1,2,10.703267,3.350650,11.619095,9.773907,5.321809
2,3,5.005981,5.594959,9.140260,5.321679,3.973032
3,4,0.000000,0.000000,4.462092,0.000000,0.000000
4,5,9.935868,5.603515,9.137877,6.999255,5.550172
...,...,...,...,...,...,...
35962,49988,8.773429,0.000000,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,7.882692,9.036106
35965,49993,0.000000,0.000000,0.000000,0.000000,8.160804


,custid,12~2시_dis_amt>=0_log_max,12시이전_dis_amt>=0_log_max,2~4시_dis_amt>=0_log_max,4~6시_dis_amt>=0_log_max,6시이후_dis_amt>=0_log_max
0,0,9.105091,0.000000,0.000000,0.000000,10.725490
1,2,10.703267,10.051951,11.619095,9.958970,10.679389
2,3,8.497195,8.560444,10.343386,8.400884,10.201776
3,4,0.000000,0.000000,9.841665,0.000000,0.000000
4,5,10.795506,11.130215,9.137877,9.923339,10.134639
...,...,...,...,...,...,...
35962,49988,9.959442,0.000000,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,7.882692,9.036106
35965,49993,0.000000,0.000000,0.000000,0.000000,8.160804


,custid,12~2시_dis_amt>=0_log_std,12시이전_dis_amt>=0_log_std,2~4시_dis_amt>=0_log_std,4~6시_dis_amt>=0_log_std,6시이후_dis_amt>=0_log_std
0,0,0.591214,0.000000,0.000000,0.000000,0.549238
1,2,0.000000,5.803496,0.000000,0.261717,6.145165
2,3,4.571776,4.848288,1.204618,4.128953,4.507210
3,4,0.000000,0.000000,5.206553,0.000000,0.000000
4,5,0.874127,4.739827,0.000000,4.358896,4.913640
...,...,...,...,...,...,...
35962,49988,1.074856,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,0.000000,0.000000


,custid,12~2시_dis_amt>=0_log_변동계수,12시이전_dis_amt>=0_log_변동계수,2~4시_dis_amt>=0_log_변동계수,4~6시_dis_amt>=0_log_변동계수,6시이후_dis_amt>=0_log_변동계수
0,0,0.068057,0.000000,0.000000,0.000000,0.056975
1,2,0.000000,1.732051,0.000000,0.026777,1.154714
2,3,0.913263,0.866546,0.131793,0.775874,1.134451
3,4,0.000000,0.000000,1.166841,0.000000,0.000000
4,5,0.087977,0.845867,0.000000,0.622766,0.885313
...,...,...,...,...,...,...
35962,49988,0.122513,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,0.000000,0.000000


**[최대구매시간, 최소구매시간, 평균구매시간]** 

In [99]:
f = data.groupby('custid')['sales_hour'].agg([('평균구매시간', np.mean),
                                            ('최대구매시간', np.max),
                                            ('최소구매시간', np.min),
                                            ('구매시간표준편차', np.std)
                                             ]).reset_index().fillna(0)
features.append(f); f

,custid,평균구매시간,최대구매시간,최소구매시간,구매시간표준편차
0,0,17.363636,19,12,2.693426
1,2,15.090909,19,10,3.645670
2,3,15.800000,19,11,2.746785
3,4,14.000000,14,14,0.000000
4,5,15.500000,19,11,3.162278
...,...,...,...,...,...
35962,49988,13.500000,18,12,3.000000
35963,49990,15.000000,15,15,0.000000
35964,49992,17.500000,18,17,0.707107
35965,49993,17.000000,18,16,0.816497


**[구매시간다양성]**

In [100]:
n = data.sales_hour.nunique()
f = data.groupby('custid')['sales_hour'].agg([('구매시간다양성', lambda x: len(x.unique())/n)]).reset_index()
features.append(f); display(f)

,custid,구매시간다양성
0,0,0.176471
1,2,0.470588
2,3,0.470588
3,4,0.058824
4,5,0.470588
...,...,...
35962,49988,0.117647
35963,49990,0.058824
35964,49992,0.117647
35965,49993,0.176471


**[상하반기구매건수, 환불건수, 그외다수]** 

In [101]:
f = pd.pivot_table(data = data, columns = '상하반기', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

상하반기,custid,상반기_구매건수,하반기_구매건수
0,0,3,8
1,2,1,8
2,3,11,17
3,4,0,4
4,5,9,23
...,...,...,...
35962,49988,1,3
35963,49990,1,0
35964,49992,1,1
35965,49993,4,0


In [102]:
f = pd.pivot_table(data = data, columns = '상하반기', index = 'custid', values = '환불건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_환불건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

상하반기,custid,상반기_환불건수,하반기_환불건수
0,0,0,0
1,2,1,1
2,3,1,1
3,4,0,0
4,5,0,0
...,...,...,...
35962,49988,0,0
35963,49990,0,0
35964,49992,0,0
35965,49993,0,0


In [103]:
cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = '상하반기', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        dic2 = {}
        g = unique_custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
        
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,상반기_tot_amt>=0_sum,하반기_tot_amt>=0_sum
0,0,423000,1319000
1,2,1200000,4401350
2,3,1840200,1155900
3,4,0,1045000
4,5,1657500,3396259
...,...,...,...
35962,49988,59000,573000
35963,49990,213000,0
35964,49992,168000,53000
35965,49993,143029,0


,custid,상반기_tot_amt>=0_mean,하반기_tot_amt>=0_mean
0,0,141000.000000,164875.000000
1,2,600000.000000,489038.888889
2,3,153350.000000,64216.666667
3,4,0.000000,261250.000000
4,5,184166.666667,147663.434783
...,...,...,...
35962,49988,59000.000000,191000.000000
35963,49990,213000.000000,0.000000
35964,49992,168000.000000,53000.000000
35965,49993,35757.250000,0.000000


,custid,상반기_tot_amt>=0_max,하반기_tot_amt>=0_max
0,0,294000,455000
1,2,1200000,1416000
2,3,621000,187000
3,4,0,560000
4,5,434000,682000
...,...,...,...
35962,49988,59000,423000
35963,49990,213000,0
35964,49992,168000,53000
35965,49993,70000,0


,custid,상반기_tot_amt>=0_std,하반기_tot_amt>=0_std
0,0,134933.316864,121220.504513
1,2,848528.137424,468963.352631
2,3,203551.862947,49414.385260
3,4,0.000000,250313.636598
4,5,113993.969139,167826.630229
...,...,...,...
35962,49988,0.000000,202230.067003
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,23470.074071,0.000000


,custid,상반기_tot_amt>=0_변동계수,하반기_tot_amt>=0_변동계수
0,0,0.956974,0.735227
1,2,1.414214,0.958949
2,3,1.327368,0.769495
3,4,0.000000,0.958138
4,5,0.618972,1.136548
...,...,...,...
35962,49988,0.000000,1.058796
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,0.656372,0.000000


...

,custid,상반기_dis_amt>=0_log_sum,하반기_dis_amt>=0_log_sum
0,0,27.662863,76.470993
1,2,0.000000,73.209363
2,3,62.401978,90.413075
3,4,0.000000,17.848366
4,5,66.602060,139.767647
...,...,...,...
35962,49988,7.989899,26.320288
35963,49990,9.273409,0.000000
35964,49992,9.036106,7.882692
35965,49993,8.160804,0.000000


,custid,상반기_dis_amt>=0_log_mean,하반기_dis_amt>=0_log_mean
0,0,9.220954,9.558874
1,2,0.000000,8.134374
2,3,5.200165,5.022949
3,4,0.000000,4.462092
4,5,7.400229,6.076854
...,...,...,...
35962,49988,7.989899,8.773429
35963,49990,9.273409,0.000000
35964,49992,9.036106,7.882692
35965,49993,2.040201,0.000000


,custid,상반기_dis_amt>=0_log_max,하반기_dis_amt>=0_log_max
0,0,10.288784,10.725490
1,2,0.000000,11.619095
2,3,10.343386,9.143239
3,4,0.000000,9.841665
4,5,10.134639,11.130215
...,...,...,...
35962,49988,7.989899,9.959442
35963,49990,9.273409,0.000000
35964,49992,9.036106,7.882692
35965,49993,8.160804,0.000000


,custid,상반기_dis_amt>=0_log_std,하반기_dis_amt>=0_log_std
0,0,1.014870,0.526407
1,2,0.000000,4.647519
2,3,4.645621,4.140673
3,4,0.000000,5.206553
4,5,4.218483,4.598513
...,...,...,...
35962,49988,0.000000,1.074856
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,4.080402,0.000000


,custid,상반기_dis_amt>=0_log_변동계수,하반기_dis_amt>=0_log_변동계수
0,0,0.110061,0.055070
1,2,0.000000,0.571343
2,3,0.893360,0.824351
3,4,0.000000,1.166841
4,5,0.570048,0.756726
...,...,...,...
35962,49988,0.000000,0.122513
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,2.000000,0.000000


**[명절기간총구매액,명절기간총구매건수]**

In [104]:
f = data.query('sales_month==10 & sales_day==[1, 2, 3, 4, 5, 6, 7, 8, 9] | sales_month==2 & sales_day==[13, 14, 15, 16, 17, 18]').groupby('custid')['tot_amt'].agg([('명절기간총구매액','sum'),
                                                                                                    ('명절기간총구매건수','count')]).reset_index()
custid = pd.DataFrame({'custid': data.custid.unique()})
custid = custid.sort_values(by = 'custid')

f = pd.merge(custid, f, how = 'left').fillna(0)
features.append(f); display(f)

,custid,명절기간총구매액,명절기간총구매건수
0,0,0.0,0.0
1,2,0.0,0.0
2,3,0.0,0.0
3,4,0.0,0.0
4,5,0.0,0.0
...,...,...,...
35962,49988,0.0,0.0
35963,49990,0.0,0.0
35964,49992,0.0,0.0
35965,49993,0.0,0.0


**[최근1달간구매금액,구매건수]** 

In [105]:
cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data.query('sales_month == 4'),
                                                  columns = 'sales_month', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'최근한달간_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
        
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,최근한달간_tot_amt>=0_sum
0,0,294000.0
1,2,0.0
2,3,227000.0
3,4,0.0
4,5,0.0
...,...,...
35962,49988,0.0
35963,49990,0.0
35964,49992,168000.0
35965,49993,0.0


,custid,최근한달간_tot_amt>=0_mean
0,0,294000.000000
1,2,0.000000
2,3,56750.000000
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,168000.000000
35965,49993,0.000000


,custid,최근한달간_tot_amt>=0_max
0,0,294000.0
1,2,0.0
2,3,89000.0
3,4,0.0
4,5,0.0
...,...,...
35962,49988,0.0
35963,49990,0.0
35964,49992,168000.0
35965,49993,0.0


,custid,최근한달간_tot_amt>=0_std
0,0,0.000000
1,2,0.000000
2,3,41120.757127
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,4_tot_amt>=0_변동계수
0,0,0.000000
1,2,0.000000
2,3,0.724595
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


...

,custid,최근한달간_dis_amt>=0_log_sum
0,0,10.288784
1,2,0.000000
2,3,24.638486
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,9.036106
35965,49993,0.000000


,custid,최근한달간_dis_amt>=0_log_mean
0,0,10.288784
1,2,0.000000
2,3,6.159621
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,9.036106
35965,49993,0.000000


,custid,최근한달간_dis_amt>=0_log_max
0,0,10.288784
1,2,0.000000
2,3,8.400884
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,9.036106
35965,49993,0.000000


,custid,최근한달간_dis_amt>=0_log_std
0,0,0.000000
1,2,0.000000
2,3,4.113087
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,4_dis_amt>=0_log_변동계수
0,0,0.000000
1,2,0.000000
2,3,0.667750
3,4,0.000000
4,5,0.000000
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


**[월별구매비율]**

In [106]:
f = data.groupby('custid')['sales_month'].agg([('1월_구매비율', lambda x: np.mean(x.isin([1]))),
                                            ('2월_구매비율', lambda x: np.mean(x.isin([2]))),
                                            ('3월_구매비율', lambda x: np.mean(x.isin([3]))),
                                            ('4월_구매비율', lambda x: np.mean(x.isin([4]))),
                                            ('5월_구매비율', lambda x: np.mean(x.isin([5]))),
                                            ('6월_구매비율', lambda x: np.mean(x.isin([6]))),
                                            ('7월_구매비율', lambda x: np.mean(x.isin([7]))),
                                            ('8월_구매비율', lambda x: np.mean(x.isin([8]))),
                                            ('9월_구매비율', lambda x: np.mean(x.isin([9]))),
                                            ('10월_구매비율', lambda x: np.mean(x.isin([10]))),
                                            ('11월_구매비율', lambda x: np.mean(x.isin([11]))),
                                            ('12월_구매비율', lambda x: np.mean(x.isin([12])))]).reset_index()
features.append(f); display(f)

,custid,1월_구매비율,2월_구매비율,3월_구매비율,4월_구매비율,5월_구매비율,6월_구매비율,7월_구매비율,8월_구매비율,9월_구매비율,10월_구매비율,11월_구매비율,12월_구매비율
0,0,0.000000,0.0,0.000,0.090909,0.000000,0.181818,0.000000,0.181818,0.272727,0.000000,0.000000,0.272727
1,2,0.181818,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.272727,0.181818,0.181818,0.090909,0.090909
2,3,0.066667,0.0,0.100,0.133333,0.066667,0.033333,0.233333,0.133333,0.100000,0.000000,0.000000,0.133333
3,4,0.000000,0.0,0.000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,0.000000,0.0,0.125,0.000000,0.062500,0.093750,0.187500,0.125000,0.156250,0.156250,0.062500,0.031250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0.250000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000
35963,49990,0.000000,1.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.0,0.000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,1.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**[계절별구매횟수, 환불건수, 그외다수]** 

In [107]:
f = pd.pivot_table(data = data, columns = 'sales_season', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

sales_season,custid,FALL_구매건수,SPRING_구매건수,SUMMER_구매건수,WINTER_구매건수
0,0,3,1,4,3
1,2,5,0,2,2
2,3,3,8,11,6
3,4,0,0,4,0
4,5,12,6,13,1
...,...,...,...,...,...
35962,49988,3,0,0,1
35963,49990,0,0,0,1
35964,49992,0,1,1,0
35965,49993,0,0,0,4


**[계절별구매횟수, 환불건수, 그외다수]** 

In [108]:
f = pd.pivot_table(data = data, columns = 'sales_season', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

sales_season,custid,FALL_구매건수,SPRING_구매건수,SUMMER_구매건수,WINTER_구매건수
0,0,3,1,4,3
1,2,5,0,2,2
2,3,3,8,11,6
3,4,0,0,4,0
4,5,12,6,13,1
...,...,...,...,...,...
35962,49988,3,0,0,1
35963,49990,0,0,0,1
35964,49992,0,1,1,0
35965,49993,0,0,0,4


In [109]:
f = pd.pivot_table(data = data, columns = 'sales_season', index = 'custid', values = '환불건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_환불건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

sales_season,custid,FALL_환불건수,SPRING_환불건수,SUMMER_환불건수,WINTER_환불건수
0,0,0,0,0,0
1,2,0,0,1,1
2,3,0,1,1,0
3,4,0,0,0,0
4,5,0,0,0,0
...,...,...,...,...,...
35962,49988,0,0,0,0
35963,49990,0,0,0,0
35964,49992,0,0,0,0
35965,49993,0,0,0,0


In [110]:
cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'sales_season', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
          
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,FALL_tot_amt>=0_sum,SPRING_tot_amt>=0_sum,SUMMER_tot_amt>=0_sum,WINTER_tot_amt>=0_sum
0,0,331000,294000,759000,358000
1,2,2146350,0,839000,2616000
2,3,134800,1693000,562800,605500
3,4,0,0,1045000,0
4,5,1656547,1073000,2266129,58083
...,...,...,...,...,...
35962,49988,573000,0,0,59000
35963,49990,0,0,0,213000
35964,49992,0,168000,53000,0
35965,49993,0,0,0,143029


,custid,FALL_tot_amt>=0_mean,SPRING_tot_amt>=0_mean,SUMMER_tot_amt>=0_mean,WINTER_tot_amt>=0_mean
0,0,110333.333333,294000.000000,189750.000000,119333.333333
1,2,429270.000000,0.000000,279666.666667,872000.000000
2,3,44933.333333,188111.111111,46900.000000,100916.666667
3,4,0.000000,0.000000,261250.000000,0.000000
4,5,138045.583333,178833.333333,174317.615385,58083.000000
...,...,...,...,...,...
35962,49988,191000.000000,0.000000,0.000000,59000.000000
35963,49990,0.000000,0.000000,0.000000,213000.000000
35964,49992,0.000000,168000.000000,53000.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,35757.250000


,custid,FALL_tot_amt>=0_max,SPRING_tot_amt>=0_max,SUMMER_tot_amt>=0_max,WINTER_tot_amt>=0_max
0,0,118000,294000,455000,166000
1,2,1112000,0,434500,1416000
2,3,61000,621000,135000,187000
3,4,0,0,560000,0
4,5,682000,252000,488000,58083
...,...,...,...,...,...
35962,49988,423000,0,0,59000
35963,49990,0,0,0,213000
35964,49992,0,168000,53000,0
35965,49993,0,0,0,70000


,custid,FALL_tot_amt>=0_std,SPRING_tot_amt>=0_std,SUMMER_tot_amt>=0_std,WINTER_tot_amt>=0_std
0,0,9291.573243,0.000000,185517.968582,40414.518843
1,2,397810.934365,0.000000,242662.488517,762857.784911
2,3,26115.384993,226007.435964,43434.757762,46731.591741
3,4,0.000000,0.000000,250313.636598,0.000000
4,5,179056.644830,59398.372593,168592.628686,0.000000
...,...,...,...,...,...
35962,49988,202230.067003,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,23470.074071


,custid,FALL_tot_amt>=0_변동계수,SPRING_tot_amt>=0_변동계수,SUMMER_tot_amt>=0_변동계수,WINTER_tot_amt>=0_변동계수
0,0,0.084214,0.000000,0.977697,0.338669
1,2,0.926715,0.000000,0.867685,0.874837
2,3,0.581203,1.201457,0.926114,0.463071
3,4,0.000000,0.000000,0.958138,0.000000
4,5,1.297083,0.332144,0.967158,0.000000
...,...,...,...,...,...
35962,49988,1.058796,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,0.656372


...

,custid,FALL_dis_amt>=0_log_sum,SPRING_dis_amt>=0_log_sum,SUMMER_dis_amt>=0_log_sum,WINTER_dis_amt>=0_log_sum
0,0,27.919026,10.288784,37.869582,28.056463
1,2,51.922127,0.000000,21.287236,0.000000
2,3,7.989899,54.001094,48.716763,42.107297
3,4,0.000000,0.000000,17.848366,0.000000
4,5,84.488299,48.750098,73.131310,0.000000
...,...,...,...,...,...
35962,49988,26.320288,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,0.000000,9.273409
35964,49992,0.000000,9.036106,7.882692,0.000000
35965,49993,0.000000,0.000000,0.000000,8.160804


,custid,FALL_dis_amt>=0_log_mean,SPRING_dis_amt>=0_log_mean,SUMMER_dis_amt>=0_log_mean,WINTER_dis_amt>=0_log_mean
0,0,9.306342,10.288784,9.467396,9.352154
1,2,10.384425,0.000000,7.095745,0.000000
2,3,2.663300,6.000122,4.059730,7.017883
3,4,0.000000,0.000000,4.462092,0.000000
4,5,7.040692,8.125016,5.625485,0.000000
...,...,...,...,...,...
35962,49988,8.773429,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,0.000000,9.273409
35964,49992,0.000000,9.036106,7.882692,0.000000
35965,49993,0.000000,0.000000,0.000000,2.040201


,custid,FALL_dis_amt>=0_log_max,SPRING_dis_amt>=0_log_max,SUMMER_dis_amt>=0_log_max,WINTER_dis_amt>=0_log_max
0,0,9.375940,10.288784,10.725490,9.717218
1,2,11.619095,0.000000,10.679389,0.000000
2,3,7.989899,10.343386,8.817446,9.143239
3,4,0.000000,0.000000,9.841665,0.000000
4,5,11.130215,10.134639,10.795506,0.000000
...,...,...,...,...,...
35962,49988,9.959442,0.000000,0.000000,7.989899
35963,49990,0.000000,0.000000,0.000000,9.273409
35964,49992,0.000000,9.036106,7.882692,0.000000
35965,49993,0.000000,0.000000,0.000000,8.160804


,custid,FALL_dis_amt>=0_log_std,SPRING_dis_amt>=0_log_std,SUMMER_dis_amt>=0_log_std,WINTER_dis_amt>=0_log_std
0,0,0.085828,0.000000,1.039523,0.316155
1,2,0.798525,0.000000,6.145200,0.000000
2,3,4.612971,4.573187,4.254562,3.462638
3,4,0.000000,0.000000,5.206553,0.000000
4,5,4.299803,3.994824,4.692964,0.000000
...,...,...,...,...,...
35962,49988,1.074856,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,4.080402


,custid,FALL_dis_amt>=0_log_변동계수,SPRING_dis_amt>=0_log_변동계수,SUMMER_dis_amt>=0_log_변동계수,WINTER_dis_amt>=0_log_변동계수
0,0,0.009223,0.000000,0.109800,0.033806
1,2,0.076896,0.000000,0.866040,0.000000
2,3,1.732051,0.762182,1.047991,0.493402
3,4,0.000000,0.000000,1.166841,0.000000
4,5,0.610707,0.491670,0.834233,0.000000
...,...,...,...,...,...
35962,49988,0.122513,0.000000,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.000000,2.000000


**[총할부개월수, 평균할부개월수, 최대할부개월수, 최소할부개월수]**

In [111]:
f = data.groupby('custid')['inst_mon'].agg([ ('총할부개월수', np.sum),
                                            ('평균할부개월수', np.mean),
                                            ('최대할부개월수', np.max),
                                            ('최소할부개월수', np.min),
                                           ('할부개월수표준편차', np.std)]).reset_index().fillna(0)

features.append(f); display(f)

,custid,총할부개월수,평균할부개월수,최대할부개월수,최소할부개월수,할부개월수표준편차
0,0,31,2.818182,3,1,0.603023
1,2,38,3.454545,12,1,3.416537
2,3,80,2.666667,5,1,1.184187
3,4,18,4.500000,10,1,4.041452
4,5,60,1.875000,3,1,1.008032
...,...,...,...,...,...,...
35962,49988,9,2.250000,3,1,0.957427
35963,49990,6,6.000000,6,6,0.000000
35964,49992,2,1.000000,1,1,0.000000
35965,49993,4,1.000000,1,1,0.000000


**[주방문요일]**

In [112]:
f = data.groupby('custid')['sales_dayofweek'].agg([('주방문요일', lambda x : x.value_counts().idxmax())]).reset_index()
f = pd.get_dummies(f)
features.append(f); display(f)

,custid,주방문요일_금,주방문요일_목,주방문요일_수,주방문요일_월,주방문요일_일,주방문요일_토,주방문요일_화
0,0,0,0,0,0,1,0,0
1,2,0,0,1,0,0,0,0
2,3,0,1,0,0,0,0,0
3,4,0,1,0,0,0,0,0
4,5,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,1
35963,49990,0,0,1,0,0,0,0
35964,49992,0,0,0,0,1,0,0
35965,49993,0,0,1,0,0,0,0


**[방문일수다양성]**

In [113]:
n = data.sales_dayofweek.nunique()
f = data.groupby('custid')['sales_dayofweek'].agg([('방문일수다양성',lambda x:len(x.unique())/n)]).reset_index()
features.append(f); display(f)

,custid,방문일수다양성
0,0,0.714286
1,2,0.428571
2,3,0.714286
3,4,0.285714
4,5,1.000000
...,...,...
35962,49988,0.142857
35963,49990,0.142857
35964,49992,0.285714
35965,49993,0.142857


**[요일별구매횟수, 환불건수, 그외다수]**

In [114]:
f = pd.pivot_table(data = data, columns = 'sales_dayofweek', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

sales_dayofweek,custid,금_구매건수,목_구매건수,수_구매건수,월_구매건수,일_구매건수,토_구매건수,화_구매건수
0,0,2,0,1,1,5,2,0
1,2,2,0,4,0,3,0,0
2,3,6,7,0,5,5,5,0
3,4,0,3,0,0,1,0,0
4,5,9,14,1,2,3,2,1
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,4
35963,49990,0,0,1,0,0,0,0
35964,49992,0,1,0,0,1,0,0
35965,49993,0,0,4,0,0,0,0


In [115]:
f = pd.pivot_table(data = data, columns = 'sales_dayofweek', index = 'custid', values = '환불건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_환불건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

sales_dayofweek,custid,금_환불건수,목_환불건수,수_환불건수,월_환불건수,일_환불건수,토_환불건수,화_환불건수
0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,1,0,0
2,3,0,0,0,1,1,0,0
3,4,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,0
35963,49990,0,0,0,0,0,0,0
35964,49992,0,0,0,0,0,0,0
35965,49993,0,0,0,0,0,0,0


In [116]:
cls = [ 'tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean',  'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'sales_dayofweek', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}요일_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
          
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,금요일_tot_amt>=0_sum,목요일_tot_amt>=0_sum,수요일_tot_amt>=0_sum,월요일_tot_amt>=0_sum,일요일_tot_amt>=0_sum,토요일_tot_amt>=0_sum,화요일_tot_amt>=0_sum
0,0,262000,0,96000,294000,460000,630000,0
1,2,2528000,0,2002350,0,1071000,0,0
2,3,898800,335200,0,489500,874600,398000,0
3,4,0,485000,0,0,560000,0,0
4,5,1975380,2019149,57812,128000,771000,78418,24000
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,632000
35963,49990,0,0,213000,0,0,0,0
35964,49992,0,168000,0,0,53000,0,0
35965,49993,0,0,143029,0,0,0,0


,custid,금요일_tot_amt>=0_mean,목요일_tot_amt>=0_mean,수요일_tot_amt>=0_mean,월요일_tot_amt>=0_mean,일요일_tot_amt>=0_mean,토요일_tot_amt>=0_mean,화요일_tot_amt>=0_mean
0,0,1.310000e+05,0.000000,96000.00,294000.000000,92000.000000,315000.0,0.0
1,2,1.264000e+06,0.000000,400470.00,0.000000,267750.000000,0.0,0.0
2,3,1.498000e+05,47885.714286,0.00,81583.333333,145766.666667,79600.0,0.0
3,4,0.000000e+00,161666.666667,0.00,0.000000,560000.000000,0.0,0.0
4,5,2.194867e+05,144224.928571,57812.00,64000.000000,257000.000000,39209.0,24000.0
...,...,...,...,...,...,...,...,...
35962,49988,0.000000e+00,0.000000,0.00,0.000000,0.000000,0.0,158000.0
35963,49990,0.000000e+00,0.000000,213000.00,0.000000,0.000000,0.0,0.0
35964,49992,0.000000e+00,168000.000000,0.00,0.000000,53000.000000,0.0,0.0
35965,49993,0.000000e+00,0.000000,35757.25,0.000000,0.000000,0.0,0.0


,custid,금요일_tot_amt>=0_max,목요일_tot_amt>=0_max,수요일_tot_amt>=0_max,월요일_tot_amt>=0_max,일요일_tot_amt>=0_max,토요일_tot_amt>=0_max,화요일_tot_amt>=0_max
0,0,166000,0,96000,294000,118000,455000,0
1,2,1416000,0,1200000,0,434500,0,0
2,3,621000,135000,0,187000,539000,94000,0
3,4,0,376000,0,0,560000,0,0
4,5,682000,425000,57812,93000,434000,58083,24000
...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,0,0,423000
35963,49990,0,0,213000,0,0,0,0
35964,49992,0,168000,0,0,53000,0,0
35965,49993,0,0,70000,0,0,0,0


,custid,금요일_tot_amt>=0_std,목요일_tot_amt>=0_std,수요일_tot_amt>=0_std,월요일_tot_amt>=0_std,일요일_tot_amt>=0_std,토요일_tot_amt>=0_std,화요일_tot_amt>=0_std
0,0,49497.474683,0.000000,0.000000,0.000000,31599.050619,197989.898732,0.000000
1,2,214960.461481,0.000000,474902.284160,0.000000,199561.394062,0.000000,0.000000
2,3,232059.819874,41927.215846,0.000000,66304.160252,200538.531626,16118.312567,0.000000
3,4,0.000000,185699.578172,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,222020.540942,101743.682511,0.000000,41012.193309,174553.716661,26691.866776,0.000000
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,177822.008387
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,23470.074071,0.000000,0.000000,0.000000,0.000000


,custid,금_tot_amt>=0_변동계수,목_tot_amt>=0_변동계수,수_tot_amt>=0_변동계수,월_tot_amt>=0_변동계수,일_tot_amt>=0_변동계수,토_tot_amt>=0_변동계수,화_tot_amt>=0_변동계수
0,0,0.377843,0.000000,0.000000,0.000000,0.343468,0.628539,0.000000
1,2,0.170064,0.000000,1.185862,0.000000,0.745327,0.000000,0.000000
2,3,1.549131,0.875568,0.000000,0.812717,1.375750,0.202491,0.000000
3,4,0.000000,1.148657,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,1.011545,0.705451,0.000000,0.640816,0.679197,0.680759,0.000000
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.125456
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,0.656372,0.000000,0.000000,0.000000,0.000000


...

,custid,금요일_dis_amt>=0_log_sum,목요일_dis_amt>=0_log_sum,수요일_dis_amt>=0_log_sum,월요일_dis_amt>=0_log_sum,일요일_dis_amt>=0_log_sum,토요일_dis_amt>=0_log_sum,화요일_dis_amt>=0_log_sum
0,0,18.886841,0.000000,9.169623,10.288784,45.293106,20.495503,0.000000
1,2,11.619095,0.000000,30.251082,0.000000,31.339186,0.000000,0.000000
2,3,26.734170,31.931887,0.000000,33.706413,35.804097,24.638486,0.000000
3,4,0.000000,17.848366,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,59.436027,102.668780,0.000000,17.298681,19.182578,0.000000,7.783641
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.310188
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,9.036106,0.000000,0.000000,7.882692,0.000000,0.000000
35965,49993,0.000000,0.000000,8.160804,0.000000,0.000000,0.000000,0.000000


,custid,금요일_dis_amt>=0_log_mean,목요일_dis_amt>=0_log_mean,수요일_dis_amt>=0_log_mean,월요일_dis_amt>=0_log_mean,일요일_dis_amt>=0_log_mean,토요일_dis_amt>=0_log_mean,화요일_dis_amt>=0_log_mean
0,0,9.443420,0.000000,9.169623,10.288784,9.058621,10.247751,0.000000
1,2,5.809547,0.000000,6.050216,0.000000,7.834797,0.000000,0.000000
2,3,4.455695,4.561698,0.000000,5.617736,5.967350,4.927697,0.000000
3,4,0.000000,5.949455,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,6.604003,7.333484,0.000000,8.649341,6.394193,0.000000,7.783641
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.577547
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,9.036106,0.000000,0.000000,7.882692,0.000000,0.000000
35965,49993,0.000000,0.000000,2.040201,0.000000,0.000000,0.000000,0.000000


,custid,금요일_dis_amt>=0_log_max,목요일_dis_amt>=0_log_max,수요일_dis_amt>=0_log_max,월요일_dis_amt>=0_log_max,일요일_dis_amt>=0_log_max,토요일_dis_amt>=0_log_max,화요일_dis_amt>=0_log_max
0,0,9.717218,0.000000,9.169623,10.288784,9.375940,10.725490,0.000000
1,2,11.619095,0.000000,10.703267,0.000000,10.679389,0.000000,0.000000
2,3,10.343386,8.817446,0.000000,9.143239,10.201776,8.400884,0.000000
3,4,0.000000,9.841665,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,11.130215,10.118639,0.000000,9.137877,10.134639,0.000000,7.783641
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.959442
35963,49990,0.000000,0.000000,9.273409,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,9.036106,0.000000,0.000000,7.882692,0.000000,0.000000
35965,49993,0.000000,0.000000,8.160804,0.000000,0.000000,0.000000,0.000000


,custid,금요일_dis_amt>=0_log_std,목요일_dis_amt>=0_log_std,수요일_dis_amt>=0_log_std,월요일_dis_amt>=0_log_std,일요일_dis_amt>=0_log_std,토요일_dis_amt>=0_log_std,화요일_dis_amt>=0_log_std
0,0,0.387209,0.000000,0.000000,0.000000,0.454013,0.675624,0.000000
1,2,8.215941,0.000000,5.537630,0.000000,5.230721,0.000000,0.000000
2,3,4.945294,4.287678,0.000000,4.370922,4.670985,4.502921,0.000000
3,4,0.000000,5.233430,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,5.006134,4.005905,0.000000,0.690895,5.564127,0.000000,0.000000
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.961088
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,4.080402,0.000000,0.000000,0.000000,0.000000


,custid,금_dis_amt>=0_log_변동계수,목_dis_amt>=0_log_변동계수,수_dis_amt>=0_log_변동계수,월_dis_amt>=0_log_변동계수,일_dis_amt>=0_log_변동계수,토_dis_amt>=0_log_변동계수,화_dis_amt>=0_log_변동계수
0,0,0.041003,0.000000,0.000000,0.000000,0.050119,0.065929,0.000000
1,2,1.414214,0.000000,0.915278,0.000000,0.667627,0.000000,0.000000
2,3,1.109882,0.939930,0.000000,0.778058,0.782757,0.913798,0.000000
3,4,0.000000,0.879649,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,0.758045,0.546249,0.000000,0.079878,0.870184,0.000000,0.000000
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.112047
35963,49990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35965,49993,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000


**[요일별방문비율]**

In [117]:
f = data.groupby('custid')['sales_dayofweek'].agg([('월요일_구매비율', lambda x : np.mean(x=="월")),
                                            ('화요일_구매비율', lambda x : np.mean(x=="화")),
                                            ('수요일_구매비율', lambda x : np.mean(x=="수")),
                                            ('목요일_구매비율', lambda x : np.mean(x=="목")),
                                            ('금요일_구매비율', lambda x : np.mean(x=="금")),
                                            ('토요일_구매비율', lambda x : np.mean(x=="토")),
                                            ('일요일_구매비율', lambda x : np.mean(x=="일"))]).reset_index()
features.append(f); display(f)

,custid,월요일_구매비율,화요일_구매비율,수요일_구매비율,목요일_구매비율,금요일_구매비율,토요일_구매비율,일요일_구매비율
0,0,0.090909,0.00000,0.090909,0.000000,0.181818,0.181818,0.454545
1,2,0.000000,0.00000,0.454545,0.000000,0.181818,0.000000,0.363636
2,3,0.200000,0.00000,0.000000,0.233333,0.200000,0.166667,0.200000
3,4,0.000000,0.00000,0.000000,0.750000,0.000000,0.000000,0.250000
4,5,0.062500,0.03125,0.031250,0.437500,0.281250,0.062500,0.093750
...,...,...,...,...,...,...,...,...
35962,49988,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
35963,49990,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.00000,0.000000,0.500000,0.000000,0.000000,0.500000
35965,49993,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000


**[주말방문비율]**

In [118]:
f = data.groupby('custid')['sales_dayofweek'].agg([('주말방문비율', lambda x: np.mean(x.isin(["토", "일"])))]).reset_index()
features.append(f); display(f)

,custid,주말방문비율
0,0,0.636364
1,2,0.363636
2,3,0.366667
3,4,0.250000
4,5,0.156250
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.500000
35965,49993,0.000000


**[주중주말구매횟수, 환불건수, 그외다수]**

In [119]:
f = pd.pivot_table(data = data, columns = 'week_split', index = 'custid', values = '구매건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_구매건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

week_split,custid,주말_구매건수,주중_구매건수
0,0,7,4
1,2,3,6
2,3,10,18
3,4,1,3
4,5,5,27
...,...,...,...
35962,49988,0,4
35963,49990,0,1
35964,49992,1,1
35965,49993,0,4


In [120]:
f = pd.pivot_table(data = data, columns = 'week_split', index = 'custid', values = '환불건수',
              aggfunc=np.sum, fill_value = 0).reset_index()
dic = {}
for i in f.iloc[:,1:].columns:
            dic[i] = f'{i}_환불건수'
f.rename(columns = dic, inplace = True)
features.append(f); 
display(f)

week_split,custid,주말_환불건수,주중_환불건수
0,0,0,0
1,2,1,1
2,3,1,1
3,4,0,0
4,5,0,0
...,...,...,...
35962,49988,0,0
35963,49990,0,0
35964,49992,0,0
35965,49993,0,0


In [121]:
cls = [ 'tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'week_split', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
          
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,주말_tot_amt>=0_sum,주중_tot_amt>=0_sum
0,0,1090000,652000
1,2,1071000,4530350
2,3,1272600,1723500
3,4,560000,485000
4,5,849418,4204341
...,...,...,...
35962,49988,0,632000
35963,49990,0,213000
35964,49992,53000,168000
35965,49993,0,143029


,custid,주말_tot_amt>=0_mean,주중_tot_amt>=0_mean
0,0,155714.285714,163000.000000
1,2,267750.000000,647192.857143
2,3,115690.909091,90710.526316
3,4,560000.000000,161666.666667
4,5,169883.600000,155716.333333
...,...,...,...
35962,49988,0.000000,158000.000000
35963,49990,0.000000,213000.000000
35964,49992,53000.000000,168000.000000
35965,49993,0.000000,35757.250000


,custid,주말_tot_amt>=0_max,주중_tot_amt>=0_max
0,0,455000,294000
1,2,434500,1416000
2,3,539000,621000
3,4,560000,376000
4,5,434000,682000
...,...,...,...
35962,49988,0,423000
35963,49990,0,213000
35964,49992,53000,168000
35965,49993,0,70000


,custid,주말_tot_amt>=0_std,주중_tot_amt>=0_std
0,0,137982.745643,93359.520136
1,2,199561.394062,579309.885718
2,3,146307.138955,136641.123201
3,4,0.000000,185699.578172
4,5,172169.944654,153523.603740
...,...,...,...
35962,49988,0.000000,177822.008387
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,0.000000,23470.074071


,custid,주말_tot_amt>=0_변동계수,주중_tot_amt>=0_변동계수
0,0,0.886128,0.572758
1,2,0.745327,0.895112
2,3,1.264638,1.506343
3,4,0.000000,1.148657
4,5,1.013458,0.985918
...,...,...,...
35962,49988,0.000000,1.125456
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,0.000000,0.656372


...

,custid,주말_dis_amt>=0_log_sum,주중_dis_amt>=0_log_sum
0,0,65.788608,38.345247
1,2,31.339186,41.870176
2,3,60.442583,92.372470
3,4,0.000000,17.848366
4,5,19.182578,187.187129
...,...,...,...
35962,49988,0.000000,34.310188
35963,49990,0.000000,9.273409
35964,49992,7.882692,9.036106
35965,49993,0.000000,8.160804


,custid,주말_dis_amt>=0_log_mean,주중_dis_amt>=0_log_mean
0,0,9.398373,9.586312
1,2,7.834797,5.981454
2,3,5.494780,4.861709
3,4,0.000000,5.949455
4,5,3.836516,6.932857
...,...,...,...
35962,49988,0.000000,8.577547
35963,49990,0.000000,9.273409
35964,49992,7.882692,9.036106
35965,49993,0.000000,2.040201


,custid,주말_dis_amt>=0_log_max,주중_dis_amt>=0_log_max
0,0,10.725490,10.288784
1,2,10.679389,11.619095
2,3,10.201776,10.343386
3,4,0.000000,9.841665
4,5,10.134639,11.130215
...,...,...,...
35962,49988,0.000000,9.959442
35963,49990,0.000000,9.273409
35964,49992,7.882692,9.036106
35965,49993,0.000000,8.160804


,custid,주말_dis_amt>=0_log_std,주중_dis_amt>=0_log_std
0,0,0.741735,0.534747
1,2,5.230721,5.630953
2,3,4.394810,4.302184
3,4,0.000000,5.233430
4,5,5.267396,4.240302
...,...,...,...
35962,49988,0.000000,0.961088
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,0.000000,4.080402


,custid,주말_dis_amt>=0_log_변동계수,주중_dis_amt>=0_log_변동계수
0,0,0.078922,0.055782
1,2,0.667627,0.941402
2,3,0.799815,0.884912
3,4,0.000000,0.879649
4,5,1.372964,0.611624
...,...,...,...
35962,49988,0.000000,0.112047
35963,49990,0.000000,0.000000
35964,49992,0.000000,0.000000
35965,49993,0.000000,2.000000


**[무이자할부건수,무이자할부비율]**

In [122]:
f = data.groupby('custid')['inst_fee'].agg([('무이자할부건수', np.sum),
                                            ('무이자할부비율', np.mean)]).reset_index()
features.append(f); display(f)

,custid,무이자할부건수,무이자할부비율
0,0,0,0.000000
1,2,3,0.272727
2,3,3,0.100000
3,4,2,0.500000
4,5,0,0.000000
...,...,...,...
35962,49988,0,0.000000
35963,49990,1,1.000000
35964,49992,0,0.000000
35965,49993,0,0.000000


**[수입상품총구매액]** 

In [123]:
cls = [ 'tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log', 'tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data[data['import_flg'] == 1], columns = 'import_flg', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'수입상품_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,수입상품_tot_amt>=0_sum
0,0,1117000.0
1,2,146000.0
2,3,0.0
3,4,60000.0
4,5,1489200.0
...,...,...
35962,49988,98000.0
35963,49990,213000.0
35964,49992,53000.0
35965,49993,0.0


,custid,수입상품_tot_amt>=0_mean
0,0,159571.428571
1,2,146000.000000
2,3,0.000000
3,4,60000.000000
4,5,248200.000000
...,...,...
35962,49988,98000.000000
35963,49990,213000.000000
35964,49992,53000.000000
35965,49993,0.000000


,custid,수입상품_tot_amt>=0_max
0,0,455000.0
1,2,146000.0
2,3,0.0
3,4,60000.0
4,5,682000.0
...,...,...
35962,49988,98000.0
35963,49990,213000.0
35964,49992,53000.0
35965,49993,0.0


,custid,수입상품_tot_amt>=0_std
0,0,138454.393385
1,2,0.000000
2,3,0.000000
3,4,0.000000
4,5,256495.769946
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,1_tot_amt>=0_변동계수
0,0,0.867664
1,2,0.000000
2,3,0.000000
3,4,0.000000
4,5,1.033424
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


...

,custid,수입상품_dis_amt>=0_log_sum
0,0,65.926045
1,2,9.588845
2,3,0.000000
3,4,8.006701
4,5,56.998222
...,...,...
35962,49988,8.497195
35963,49990,9.273409
35964,49992,7.882692
35965,49993,0.000000


,custid,수입상품_dis_amt>=0_log_mean
0,0,9.418006
1,2,9.588845
2,3,0.000000
3,4,8.006701
4,5,9.499704
...,...,...
35962,49988,8.497195
35963,49990,9.273409
35964,49992,7.882692
35965,49993,0.000000


,custid,수입상품_dis_amt>=0_log_max
0,0,10.725490
1,2,9.588845
2,3,0.000000
3,4,8.006701
4,5,11.130215
...,...,...
35962,49988,8.497195
35963,49990,9.273409
35964,49992,7.882692
35965,49993,0.000000


,custid,수입상품_dis_amt>=0_log_std
0,0,0.759879
1,2,0.000000
2,3,0.000000
3,4,0.000000
4,5,1.033979
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,1_dis_amt>=0_log_변동계수
0,0,0.080684
1,2,0.000000
2,3,0.000000
3,4,0.000000
4,5,0.108843
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


**[수입상품구매건수]**

In [124]:
f = pd.merge(unique_custid, pd.pivot_table(data = data[data['import_flg'] == 1], columns = 'import_flg', index = 'custid', values = '구매건수',
                      aggfunc = np.sum, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
f.rename(columns = {1: '수입상품구매건수'}, inplace = True)
features.append(f);  display(f)

,custid,수입상품구매건수
0,0,7.0
1,2,1.0
2,3,0.0
3,4,1.0
4,5,6.0
...,...,...
35962,49988,1.0
35963,49990,1.0
35964,49992,1.0
35965,49993,0.0


**[수입상품환불건수]**

In [125]:
f = pd.merge(unique_custid, pd.pivot_table(data = data[data['import_flg'] == 1], columns = 'import_flg', index = 'custid', values = '환불건수',
                      aggfunc = np.sum, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
f.rename(columns = {1: '수입상품환불건수'}, inplace = True)
features.append(f);  display(f)

,custid,수입상품환불건수
0,0,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
...,...,...
35962,49988,0.0
35963,49990,0.0
35964,49992,0.0
35965,49993,0.0


**[수입상품구매비율]**

In [126]:
x = data[data['import_flg'] == 1].groupby('custid').size() / data.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품구매비율'}).fillna(0)
features.append(f); display(f)

,custid,수입상품구매비율
0,0,0.636364
1,2,0.090909
2,3,0.000000
3,4,0.250000
4,5,0.187500
...,...,...
35962,49988,0.250000
35963,49990,1.000000
35964,49992,0.500000
35965,49993,0.000000


**[매장이용다양성]**

In [127]:
n = 4
f = data.groupby('custid')['str_nm'].agg([('매장이용다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f); display(f)

,custid,매장이용다양성
0,0,0.50
1,2,0.50
2,3,0.50
3,4,0.25
4,5,1.00
...,...,...
35962,49988,0.25
35963,49990,0.25
35964,49992,0.25
35965,49993,0.25


**[구입지점]** 

In [128]:
cls = [ 'tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean',  'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'str_nm', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
          
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,무역점_tot_amt>=0_sum,본점_tot_amt>=0_sum,신촌점_tot_amt>=0_sum,천호점_tot_amt>=0_sum
0,0,460000,1282000,0,0
1,2,5012000,0,0,589350
2,3,462500,0,0,2533600
3,4,1045000,0,0,0
4,5,552000,3858459,391300,252000
...,...,...,...,...,...
35962,49988,0,632000,0,0
35963,49990,213000,0,0,0
35964,49992,0,221000,0,0
35965,49993,0,0,143029,0


,custid,무역점_tot_amt>=0_mean,본점_tot_amt>=0_mean,신촌점_tot_amt>=0_mean,천호점_tot_amt>=0_mean
0,0,92000.0,213666.666667,0.00,0.000000
1,2,626500.0,0.000000,0.00,196450.000000
2,3,115625.0,0.000000,0.00,97446.153846
3,4,261250.0,0.000000,0.00,0.000000
4,5,184000.0,160769.125000,97825.00,252000.000000
...,...,...,...,...,...
35962,49988,0.0,158000.000000,0.00,0.000000
35963,49990,213000.0,0.000000,0.00,0.000000
35964,49992,0.0,110500.000000,0.00,0.000000
35965,49993,0.0,0.000000,35757.25,0.000000


,custid,무역점_tot_amt>=0_max,본점_tot_amt>=0_max,신촌점_tot_amt>=0_max,천호점_tot_amt>=0_max
0,0,118000,455000,0,0
1,2,1416000,0,0,232000
2,3,187000,0,0,621000
3,4,560000,0,0,0
4,5,248000,682000,200000,252000
...,...,...,...,...,...
35962,49988,0,423000,0,0
35963,49990,213000,0,0,0
35964,49992,0,168000,0,0
35965,49993,0,0,70000,0


,custid,무역점_tot_amt>=0_std,본점_tot_amt>=0_std,신촌점_tot_amt>=0_std,천호점_tot_amt>=0_std
0,0,31599.050619,138674.679256,0.000000,0.000000
1,2,546680.567477,0.000000,0.000000,44894.403883
2,3,50697.427614,0.000000,0.000000,147732.615846
3,4,250313.636598,0.000000,0.000000,0.000000
4,5,57166.423712,172632.011740,70033.248056,0.000000
...,...,...,...,...,...
35962,49988,0.000000,177822.008387,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,81317.279836,0.000000,0.000000
35965,49993,0.000000,0.000000,23470.074071,0.000000


,custid,무역점_tot_amt>=0_변동계수,본점_tot_amt>=0_변동계수,신촌점_tot_amt>=0_변동계수,천호점_tot_amt>=0_변동계수
0,0,0.343468,0.649023,0.000000,0.000000
1,2,0.872595,0.000000,0.000000,0.228528
2,3,0.438464,0.000000,0.000000,1.516044
3,4,0.958138,0.000000,0.000000,0.000000
4,5,0.310687,1.073788,0.715903,0.000000
...,...,...,...,...,...
35962,49988,0.000000,1.125456,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.735903,0.000000,0.000000
35965,49993,0.000000,0.000000,0.656372,0.000000


...

,custid,무역점_dis_amt>=0_log_sum,본점_dis_amt>=0_log_sum,신촌점_dis_amt>=0_log_sum,천호점_dis_amt>=0_log_sum
0,0,45.293106,58.840750,0.000000,0.000000
1,2,43.609597,0.000000,0.000000,29.599766
2,3,33.706413,0.000000,0.000000,119.108640
3,4,17.848366,0.000000,0.000000,0.000000
4,5,19.651136,158.731970,17.851962,10.134639
...,...,...,...,...,...
35962,49988,0.000000,34.310188,0.000000,0.000000
35963,49990,9.273409,0.000000,0.000000,0.000000
35964,49992,0.000000,16.918798,0.000000,0.000000
35965,49993,0.000000,0.000000,8.160804,0.000000


,custid,무역점_dis_amt>=0_log_mean,본점_dis_amt>=0_log_mean,신촌점_dis_amt>=0_log_mean,천호점_dis_amt>=0_log_mean
0,0,9.058621,9.806792,0.000000,0.000000
1,2,5.451200,0.000000,0.000000,9.866589
2,3,8.426603,0.000000,0.000000,4.581102
3,4,4.462092,0.000000,0.000000,0.000000
4,5,6.550379,6.613832,4.462991,10.134639
...,...,...,...,...,...
35962,49988,0.000000,8.577547,0.000000,0.000000
35963,49990,9.273409,0.000000,0.000000,0.000000
35964,49992,0.000000,8.459399,0.000000,0.000000
35965,49993,0.000000,0.000000,2.040201,0.000000


,custid,무역점_dis_amt>=0_log_max,본점_dis_amt>=0_log_max,신촌점_dis_amt>=0_log_max,천호점_dis_amt>=0_log_max
0,0,9.375940,10.725490,0.000000,0.000000
1,2,11.619095,0.000000,0.000000,10.051951
2,3,9.143239,0.000000,0.000000,10.343386
3,4,9.841665,0.000000,0.000000,0.000000
4,5,10.118639,11.130215,8.942592,10.134639
...,...,...,...,...,...
35962,49988,0.000000,9.959442,0.000000,0.000000
35963,49990,9.273409,0.000000,0.000000,0.000000
35964,49992,0.000000,9.036106,0.000000,0.000000
35965,49993,0.000000,0.000000,8.160804,0.000000


,custid,무역점_dis_amt>=0_log_std,본점_dis_amt>=0_log_std,신촌점_dis_amt>=0_log_std,천호점_dis_amt>=0_log_std
0,0,0.454013,0.616063,0.000000,0.000000
1,2,5.836026,0.000000,0.000000,0.244984
2,3,0.531648,0.000000,0.000000,4.367980
3,4,5.206553,0.000000,0.000000,0.000000
4,5,5.680360,4.396947,5.153435,0.000000
...,...,...,...,...,...
35962,49988,0.000000,0.961088,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.815587,0.000000,0.000000
35965,49993,0.000000,0.000000,4.080402,0.000000


,custid,무역점_dis_amt>=0_log_변동계수,본점_dis_amt>=0_log_변동계수,신촌점_dis_amt>=0_log_변동계수,천호점_dis_amt>=0_log_변동계수
0,0,0.050119,0.062820,0.000000,0.000000
1,2,1.070595,0.000000,0.000000,0.024830
2,3,0.063092,0.000000,0.000000,0.953478
3,4,1.166841,0.000000,0.000000,0.000000
4,5,0.867180,0.664811,1.154705,0.000000
...,...,...,...,...,...
35962,49988,0.000000,0.112047,0.000000,0.000000
35963,49990,0.000000,0.000000,0.000000,0.000000
35964,49992,0.000000,0.096412,0.000000,0.000000
35965,49993,0.000000,0.000000,2.000000,0.000000


**[팀별]** 

In [129]:
cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log','tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean',  'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data, columns = 'team_nm', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'{i}_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
            
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,식품팀_tot_amt>=0_sum,의류패션팀_tot_amt>=0_sum,잡화가용팀_tot_amt>=0_sum
0,0,0,231000,1511000
1,2,0,1516000,4085350
2,3,25800,1441800,1528500
3,4,0,985000,60000
4,5,343859,1620400,3089500
...,...,...,...,...
35962,49988,0,475000,157000
35963,49990,0,0,213000
35964,49992,0,168000,53000
35965,49993,53029,0,90000


,custid,식품팀_tot_amt>=0_mean,의류패션팀_tot_amt>=0_mean,잡화가용팀_tot_amt>=0_mean
0,0,0.0,115500.000000,167888.888889
1,2,0.0,303200.000000,680891.666667
2,3,8600.0,110907.692308,109178.571429
3,4,0.0,328333.333333,60000.000000
4,5,68771.8,162040.000000,181735.294118
...,...,...,...,...
35962,49988,0.0,237500.000000,78500.000000
35963,49990,0.0,0.000000,213000.000000
35964,49992,0.0,168000.000000,53000.000000
35965,49993,26514.5,0.000000,45000.000000


,custid,식품팀_tot_amt>=0_max,의류패션팀_tot_amt>=0_max,잡화가용팀_tot_amt>=0_max
0,0,0,118000,455000
1,2,0,445000,1416000
2,3,14800,621000,539000
3,4,0,560000,60000
4,5,166049,488000,682000
...,...,...,...,...
35962,49988,0,423000,98000
35963,49990,0,0,213000
35964,49992,0,168000,53000
35965,49993,32029,0,70000


,custid,식품팀_tot_amt>=0_std,의류패션팀_tot_amt>=0_std,잡화가용팀_tot_amt>=0_std
0,0,0.000000,3535.533906,130391.184944
1,2,0.000000,190136.990089,627041.314761
2,3,5524.490927,158312.215056,131815.081305
3,4,0.000000,258813.317535,0.000000
4,5,56530.387587,129107.080450,180255.814543
...,...,...,...,...
35962,49988,0.000000,262336.615820,27577.164466
35963,49990,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000
35965,49993,7798.680690,0.000000,35355.339059


,custid,식품팀_tot_amt>=0_변동계수,의류패션팀_tot_amt>=0_변동계수,잡화가용팀_tot_amt>=0_변동계수
0,0,0.000000,0.030611,0.776652
1,2,0.000000,0.627101,0.920912
2,3,0.642383,1.427423,1.207335
3,4,0.000000,0.788264,0.000000
4,5,0.822000,0.796761,0.991859
...,...,...,...,...
35962,49988,0.000000,1.104575,0.351301
35963,49990,0.000000,0.000000,0.000000
35964,49992,0.000000,0.000000,0.000000
35965,49993,0.294129,0.000000,0.785674


...

,custid,식품팀_dis_amt>=0_log_sum,의류패션팀_dis_amt>=0_log_sum,잡화가용팀_dis_amt>=0_log_sum
0,0,0.0,18.708586,85.425270
1,2,0.0,42.042453,31.166910
2,3,0.0,60.106490,92.708563
3,4,0.0,9.841665,8.006701
4,5,0.0,84.680694,121.689013
...,...,...,...,...
35962,49988,0.0,17.823094,16.487094
35963,49990,0.0,0.000000,9.273409
35964,49992,0.0,9.036106,7.882692
35965,49993,0.0,0.000000,8.160804


,custid,식품팀_dis_amt>=0_log_mean,의류패션팀_dis_amt>=0_log_mean,잡화가용팀_dis_amt>=0_log_mean
0,0,0.0,9.354293,9.491697
1,2,0.0,8.408491,5.194485
2,3,0.0,4.623576,6.622040
3,4,0.0,3.280555,8.006701
4,5,0.0,8.468069,7.158177
...,...,...,...,...
35962,49988,0.0,8.911547,8.243547
35963,49990,0.0,0.000000,9.273409
35964,49992,0.0,9.036106,7.882692
35965,49993,0.0,0.000000,4.080402


,custid,식품팀_dis_amt>=0_log_max,의류패션팀_dis_amt>=0_log_max,잡화가용팀_dis_amt>=0_log_max
0,0,0.0,9.375940,10.725490
1,2,0.0,10.703267,11.619095
2,3,0.0,10.343386,10.201776
3,4,0.0,9.841665,8.006701
4,5,0.0,10.795506,11.130215
...,...,...,...,...
35962,49988,0.0,9.959442,8.497195
35963,49990,0.0,0.000000,9.273409
35964,49992,0.0,9.036106,7.882692
35965,49993,0.0,0.000000,8.160804


,custid,식품팀_dis_amt>=0_log_std,의류패션팀_dis_amt>=0_log_std,잡화가용팀_dis_amt>=0_log_std
0,0,0.0,0.030613,0.726030
1,2,0.0,4.708073,5.731219
2,3,0.0,4.496988,3.648044
3,4,0.0,5.682088,0.000000
4,5,0.0,3.074711,4.157551
...,...,...,...,...
35962,49988,0.0,1.481948,0.358712
35963,49990,0.0,0.000000,0.000000
35964,49992,0.0,0.000000,0.000000
35965,49993,0.0,0.000000,5.770560


,custid,식품팀_dis_amt>=0_log_변동계수,의류패션팀_dis_amt>=0_log_변동계수,잡화가용팀_dis_amt>=0_log_변동계수
0,0,0.0,0.003273,0.076491
1,2,0.0,0.559919,1.103328
2,3,0.0,0.972621,0.550894
3,4,0.0,1.732051,0.000000
4,5,0.0,0.363095,0.580811
...,...,...,...,...
35962,49988,0.0,0.166295,0.043514
35963,49990,0.0,0.000000,0.000000
35964,49992,0.0,0.000000,0.000000
35965,49993,0.0,0.000000,1.414214


### Add Features Round2 

In [130]:
# 백화점 영업시간 내 구매

cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log', 'tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data = data[data['sales_time_open'] == 1], 
                                                  columns = 'sales_time_open', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'영업시간내_{cl}_{agg}'    
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)

,custid,영업시간내_tot_amt>=0_sum
0,0,1742000.0
1,2,4401350.0
2,3,2996100.0
3,4,1045000.0
4,5,5053759.0
...,...,...
35962,49988,632000.0
35963,49990,213000.0
35964,49992,221000.0
35965,49993,143029.0


,custid,영업시간내_tot_amt>=0_mean
0,0,158363.636364
1,2,489038.888889
2,3,99870.000000
3,4,261250.000000
4,5,157929.968750
...,...,...
35962,49988,158000.000000
35963,49990,213000.000000
35964,49992,110500.000000
35965,49993,35757.250000


,custid,영업시간내_tot_amt>=0_max
0,0,455000.0
1,2,1416000.0
2,3,621000.0
3,4,560000.0
4,5,682000.0
...,...,...
35962,49988,423000.0
35963,49990,213000.0
35964,49992,168000.0
35965,49993,70000.0


,custid,영업시간내_tot_amt>=0_std
0,0,118540.518581
1,2,468963.352631
2,3,138275.021182
3,4,250313.636598
4,5,153688.376617
...,...,...
35962,49988,177822.008387
35963,49990,0.000000
35964,49992,81317.279836
35965,49993,23470.074071


,custid,영업시간내_dis_amt>=0_sum
0,0,174200.0
1,2,298540.0
2,3,126440.0
3,4,21800.0
4,5,361260.0
...,...,...
35962,49988,31600.0
35963,49990,10650.0
35964,49992,11050.0
35965,49993,3500.0


...

,custid,영업시간내_tot_amt>=0_log_std
0,0,0.651878
1,2,4.374982
2,3,2.987954
3,4,1.246449
4,5,0.888755
...,...,...
35962,49988,0.961228
35963,49990,0.000000
35964,49992,0.815760
35965,49993,0.579680


,custid,영업시간내_dis_amt>=0_log_sum
0,0,104.133856
1,2,73.209363
2,3,152.815053
3,4,17.848366
4,5,206.369707
...,...,...
35962,49988,34.310188
35963,49990,9.273409
35964,49992,16.918798
35965,49993,8.160804


,custid,영업시간내_dis_amt>=0_log_mean
0,0,9.466714
1,2,8.134374
2,3,5.093835
3,4,4.462092
4,5,6.449053
...,...,...
35962,49988,8.577547
35963,49990,9.273409
35964,49992,8.459399
35965,49993,2.040201


,custid,영업시간내_dis_amt>=0_log_max
0,0,10.725490
1,2,11.619095
2,3,10.343386
3,4,9.841665
4,5,11.130215
...,...,...
35962,49988,9.959442
35963,49990,9.273409
35964,49992,9.036106
35965,49993,8.160804


,custid,영업시간내_dis_amt>=0_log_std
0,0,0.651828
1,2,4.647519
2,3,4.271371
3,4,5.206553
4,5,4.468213
...,...,...
35962,49988,0.961088
35963,49990,0.000000
35964,49992,0.815587
35965,49993,4.080402


In [131]:
# 방문주기에 대한 계산

f = data.groupby('custid')['diff_time'].agg([('방문주기총합', np.sum),
                                            ('방문주기평균', np.mean),
                                            ('최대방문주기', np.max),
                                            ('방문주기표준편차', np.std),   
                                            ('방문주기변동계수', lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))]).reset_index().fillna(0)
features.append(f); display(f)

,custid,방문주기총합,방문주기평균,최대방문주기,방문주기표준편차,방문주기변동계수
0,0,284,25.818182,101,41.062923,1.516449
1,2,126,11.454545,33,12.428706,1.034551
2,3,324,10.800000,114,24.832404,2.260650
3,4,2,0.500000,2,1.000000,1.732051
4,5,314,9.812500,106,19.439629,1.949908
...,...,...,...,...,...,...
35962,49988,56,14.000000,56,28.000000,1.732051
35963,49990,0,0.000000,0,0.000000,0.000000
35964,49992,284,142.000000,284,200.818326,1.000000
35965,49993,0,0.000000,0,0.000000,0.000000


In [132]:
# 로그 변환 피쳐 계산

f = data.groupby('custid')['dis_amt>=0_log'].agg([('총할인액_log', np.sum),
                                            ('평균총할인액_log', np.mean),
                                            ('최대총할인액_log', np.max),
                                            ('최소총할인액_log', np.min),
                                            ('할인액표준편차_log', np.std),   
                                            ('할인액변동계수_log', lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))]).reset_index().fillna(0)
features.append(f); display(f)

,custid,총할인액_log,평균총할인액_log,최대총할인액_log,최소총할인액_log,할인액표준편차_log,할인액변동계수_log
0,0,104.133856,9.466714,10.725490,8.268988,0.651828,0.065650
1,2,73.209363,6.655397,11.619095,0.000000,5.301608,0.759517
2,3,152.815053,5.093835,10.343386,0.000000,4.271371,0.824443
3,4,17.848366,4.462092,9.841665,0.000000,5.206553,1.010514
4,5,206.369707,6.449053,11.130215,0.000000,4.468213,0.681936
...,...,...,...,...,...,...,...
35962,49988,34.310188,8.577547,9.959442,7.863651,0.961088,0.097035
35963,49990,9.273409,9.273409,9.273409,9.273409,0.000000,0.000000
35964,49992,16.918798,8.459399,9.036106,7.882692,0.815587,0.068174
35965,49993,8.160804,2.040201,8.160804,0.000000,4.080402,1.732051


In [133]:
# 할부혜택을 받은 사람들 피쳐 계산

cls = ['tot_amt>=0', 'dis_amt>=0', 'inst_mon', 'refund_tot_amt_log', 'tot_amt>=0_log', 'dis_amt>=0_log']
for cl in cls:   
    aggs = ['sum', 'mean', 'max', 'std']
    for agg in aggs:
        f =pd.merge(unique_custid, pd.pivot_table(data =data[data['card_friendly'].fillna(0) == 1], 
                                                  columns = 'card_friendly', index = 'custid', values = cl,
                      aggfunc = agg, fill_value = 0).reset_index(), how = 'left', on = 'custid').fillna(0)
        dic = {}
        for i in f.iloc[:,1:].columns:
                    dic[i] = f'할부혜택_{cl}_{agg}'    
        dic2 = {}
        g = custid
        if agg == 'mean':
            me = f.iloc[: , 1:]
        if agg == 'std':
            st = f.iloc[: , 1:]
       
        f.rename(columns = dic, inplace = True)
        features.append(f); 
        display(f)
    g = pd.concat([g ,(st / me).fillna(0)], axis = 1)
    for t in g.iloc[:,1:].columns:
        dic2[t] = f'{t}_{cl}_변동계수'
    g.rename(columns = dic2, inplace = True)
    features.append(g);
    display(g);

,custid,할부혜택_tot_amt>=0_sum
0,0,1703000.0
1,2,888350.0
2,3,1647300.0
3,4,60000.0
4,5,3608000.0
...,...,...
35962,49988,580000.0
35963,49990,0.0
35964,49992,0.0
35965,49993,0.0


,custid,할부혜택_tot_amt>=0_mean
0,0,170300.000000
1,2,296116.666667
2,3,91516.666667
3,4,60000.000000
4,5,257714.285714
...,...,...
35962,49988,193333.333333
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,할부혜택_tot_amt>=0_max
0,0,455000.0
1,2,445000.0
2,3,187000.0
3,4,60000.0
4,5,682000.0
...,...,...
35962,49988,423000.0
35963,49990,0.0
35964,49992,0.0
35965,49993,0.0


,custid,할부혜택_tot_amt>=0_std
0,0,117778.558698
1,2,129349.491044
2,3,35159.004369
3,4,0.000000
4,5,180213.304628
...,...,...
35962,49988,199850.777665
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,1.0_tot_amt>=0_변동계수
0,0,0.691595
1,2,0.436819
2,3,0.384181
3,4,0.000000
4,5,0.699276
...,...,...
35962,49988,1.033711
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


...

,custid,할부혜택_dis_amt>=0_log_sum
0,0,95.864868
1,2,30.714187
2,3,109.155450
3,4,8.006701
4,5,108.235701
...,...,...
35962,49988,26.446536
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,할부혜택_dis_amt>=0_log_mean
0,0,9.586487
1,2,10.238062
2,3,6.064192
3,4,8.006701
4,5,7.731122
...,...,...
35962,49988,8.815512
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,할부혜택_dis_amt>=0_log_max
0,0,10.725490
1,2,10.703267
2,3,9.143239
3,4,8.006701
4,5,11.130215
...,...,...
35962,49988,9.959442
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,할부혜택_dis_amt>=0_log_std
0,0,0.544752
1,2,0.405552
2,3,3.882735
3,4,0.000000
4,5,4.232670
...,...,...
35962,49988,1.022629
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


,custid,1.0_dis_amt>=0_log_변동계수
0,0,0.056825
1,2,0.039612
2,3,0.640272
3,4,0.000000
4,5,0.547485
...,...,...
35962,49988,0.116003
35963,49990,0.000000
35964,49992,0.000000
35965,49993,0.000000


In [134]:
# 쇼핑간격

f = data.groupby('custid')['shopping_gap_open'].agg([('쇼핑간격총합', np.sum),
                                            ('쇼핑간격평균', np.mean),
                                            ('최대쇼핑간격', np.max),
                                            ('쇼핑간격표준편차', np.std),   
                                            ('쇼핑간격변동계수', lambda x : np.std(x) / 0.0001 \
                                                                 if np.mean(x) == 0 else np.std(x) / np.mean(x))]).reset_index().fillna(0)
features.append(f); display(f)

,custid,쇼핑간격총합,쇼핑간격평균,최대쇼핑간격,쇼핑간격표준편차,쇼핑간격변동계수
0,0,138.0,12.545455,48.0,18.635255,1.416291
1,2,50.0,4.545455,30.0,10.357255,2.172556
2,3,363.0,12.100000,50.0,14.179514,1.152164
3,4,47.0,11.750000,40.0,19.120234,1.409243
4,5,178.0,5.562500,40.0,9.728367,1.721376
...,...,...,...,...,...,...
35962,49988,20.0,5.000000,10.0,5.773503,1.000000
35963,49990,0.0,0.000000,0.0,0.000000,0.000000
35964,49992,0.0,0.000000,0.0,0.000000,0.000000
35965,49993,123.0,30.750000,43.0,20.548723,0.578722


In [136]:
age = pd.read_csv(os.path.abspath("../input")+"/y_train.csv" , encoding = 'cp949')
train_age = pd.merge(train, age, how='left')

In [137]:
train_age['age'] = train_age['group'].str[1:]
train_age['age'] = train_age['age'].astype(int)

In [138]:
train['age_group'] = train_age['age'].apply(lambda x : 'twenty' if (x>=20) & (x<30)
                                     else 'thirty' if (x>=30) & (x<40)
                                     else 'forty' if (x>=40) & (x<50)
                                     else 'fifty' if (x>=50) & (x<60)
                                     else 'sixty')

twenty_prefer_brd = train[train['age_group'] == 'twenty'].brd_nm.value_counts().index[1:].to_list()
thirty_prefer_brd = train[train['age_group'] == 'thirty'].brd_nm.value_counts().index[1:].to_list()
forty_prefer_brd = train[train['age_group'] == 'forty'].brd_nm.value_counts().index[1:].to_list()
fifty_prefer_brd = train[train['age_group'] == 'fifty'].brd_nm.value_counts().index[1:].to_list()
sixty_prefer_brd = train[train['age_group'] == 'sixty'].brd_nm.value_counts().index[1:].to_list()

def prefer_brd(x, list):
    for i in range(len(list)):
        if(x == list[i]):
            return len(list)-i

data['20_weight'] = data['brd_nm'].apply(lambda x: prefer_brd(x, twenty_prefer_brd)).fillna(0)
data['30_weight'] = data['brd_nm'].apply(lambda x: prefer_brd(x, thirty_prefer_brd)).fillna(0)
data['40_weight'] = data['brd_nm'].apply(lambda x: prefer_brd(x, forty_prefer_brd)).fillna(0)
data['50_weight'] = data['brd_nm'].apply(lambda x: prefer_brd(x, fifty_prefer_brd)).fillna(0)
data['60_weight'] = data['brd_nm'].apply(lambda x: prefer_brd(x, sixty_prefer_brd)).fillna(0)

data['weight_sum'] = data['20_weight'] + data['30_weight'] + data['40_weight'] + data['50_weight'] + data['60_weight']

In [139]:
f = data.groupby('custid')['20_weight', '30_weight', '40_weight', '50_weight', '60_weight', 'weight_sum'].sum()

f['20_weight_ratio'] = f['20_weight'] / f['weight_sum']
f['30_weight_ratio'] = f['30_weight'] / f['weight_sum'] 
f['40_weight_ratio'] = f['40_weight'] / f['weight_sum'] 
f['50_weight_ratio'] = f['50_weight'] / f['weight_sum'] 
f['60_weight_ratio'] = f['60_weight'] / f['weight_sum'] 

f = f.reset_index().fillna(0)
features.append(f); display(f)

C:\Users\choij\AppData\Local\Temp\ipykernel_16976\635687817.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  f = data.groupby('custid')['20_weight', '30_weight', '40_weight', '50_weight', '60_weight', 'weight_sum'].sum()


,custid,20_weight,30_weight,40_weight,50_weight,60_weight,weight_sum,20_weight_ratio,30_weight_ratio,40_weight_ratio,50_weight_ratio,60_weight_ratio
0,0,16456.0,17869.0,17114.0,16518.0,0,67957.0,0.242153,0.262946,0.251836,0.243065,0.0
1,2,16214.0,17463.0,16996.0,16082.0,0,66755.0,0.242888,0.261598,0.254603,0.240911,0.0
2,3,35830.0,38894.0,38142.0,33518.0,0,146384.0,0.244767,0.265698,0.260561,0.228973,0.0
3,4,4715.0,4911.0,4597.0,4211.0,0,18434.0,0.255777,0.266410,0.249376,0.228437,0.0
4,5,36609.0,41256.0,40150.0,38261.0,0,156276.0,0.234259,0.263994,0.256917,0.244830,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,4703.0,6044.0,5579.0,4630.0,0,20956.0,0.224423,0.288414,0.266224,0.220939,0.0
35963,49990,1582.0,1706.0,1653.0,1554.0,0,6495.0,0.243572,0.262664,0.254503,0.239261,0.0
35964,49992,2682.0,2983.0,2913.0,2833.0,0,11411.0,0.235036,0.261414,0.255280,0.248269,0.0
35965,49993,2549.0,3274.0,2907.0,2245.0,0,10975.0,0.232255,0.298314,0.264875,0.204556,0.0


**[연령대별 브랜드구매건수, 구매비율]**

In [140]:
best_brd_20 = list(train_age.query('20<=age<30').brd_nm.value_counts().index[:300])

f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index()
brd_20 = data.query('brd_nm == @best_brd_20').groupby('custid')['구매건수'].agg([('20대브랜드_구매건수', np.sum)]).reset_index()

f = pd.merge(f, brd_20, on = 'custid', how = 'left').fillna(0)
f['20대브랜드_구매비율'] = f['20대브랜드_구매건수'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
features.append(f); display(f)

,custid,20대브랜드_구매건수,20대브랜드_구매비율
0,0,9.0,0.818182
1,2,9.0,1.000000
2,3,22.0,0.785714
3,4,1.0,0.250000
4,5,24.0,0.750000
...,...,...,...
35962,49988,1.0,0.250000
35963,49990,1.0,1.000000
35964,49992,1.0,0.500000
35965,49993,3.0,0.750000


In [141]:
best_brd_30 = list(train_age.query('30<=age<40').brd_nm.value_counts().index[:300])

f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index()
brd_30 = data.query('brd_nm == @best_brd_30').groupby('custid')['구매건수'].agg([('30대브랜드_구매건수', np.sum)]).reset_index()

f = pd.merge(f, brd_30, on = 'custid', how = 'left').fillna(0)
f['30대브랜드_구매비율'] = f['30대브랜드_구매건수'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
features.append(f); display(f)

,custid,30대브랜드_구매건수,30대브랜드_구매비율
0,0,10.0,0.909091
1,2,9.0,1.000000
2,3,21.0,0.750000
3,4,1.0,0.250000
4,5,26.0,0.812500
...,...,...,...
35962,49988,3.0,0.750000
35963,49990,1.0,1.000000
35964,49992,1.0,0.500000
35965,49993,4.0,1.000000


In [142]:
best_brd_40 = list(train_age.query('40<=age<50').brd_nm.value_counts().index[:300])

f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index()
brd_40 = data.query('brd_nm == @best_brd_40').groupby('custid')['구매건수'].agg([('40대브랜드_구매건수', np.sum)]).reset_index()

f = pd.merge(f, brd_40, on = 'custid', how = 'left').fillna(0)
f['40대브랜드_구매비율'] = f['40대브랜드_구매건수'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
features.append(f); display(f)

,custid,40대브랜드_구매건수,40대브랜드_구매비율
0,0,9.0,0.818182
1,2,8.0,0.888889
2,3,20.0,0.714286
3,4,1.0,0.250000
4,5,27.0,0.843750
...,...,...,...
35962,49988,2.0,0.500000
35963,49990,1.0,1.000000
35964,49992,1.0,0.500000
35965,49993,3.0,0.750000


In [143]:
best_brd_50 = list(train_age.query('50<=age<60').brd_nm.value_counts().index[:300])

f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index()
brd_50 = data.query('brd_nm == @best_brd_50').groupby('custid')['구매건수'].agg([('50대브랜드_구매건수', np.sum)]).reset_index()

f = pd.merge(f, brd_50, on = 'custid', how = 'left').fillna(0)
f['50대브랜드_구매비율'] = f['50대브랜드_구매건수'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
features.append(f); display(f)

,custid,50대브랜드_구매건수,50대브랜드_구매비율
0,0,10.0,0.909091
1,2,9.0,1.000000
2,3,19.0,0.678571
3,4,1.0,0.250000
4,5,29.0,0.906250
...,...,...,...
35962,49988,2.0,0.500000
35963,49990,1.0,1.000000
35964,49992,2.0,1.000000
35965,49993,3.0,0.750000


In [144]:
best_brd_60 = list(train_age.query('60<=age<70').brd_nm.value_counts().index[:300])

f = data.groupby('custid')['구매건수'].agg([('구매건수', np.sum)]).reset_index()
brd_60 = data.query('brd_nm == @best_brd_60').groupby('custid')['구매건수'].agg([('60대브랜드_구매건수', np.sum)]).reset_index()

f = pd.merge(f, brd_60, on = 'custid', how = 'left').fillna(0)
f['60대브랜드_구매비율'] = f['60대브랜드_구매건수'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
features.append(f); display(f)

,custid,60대브랜드_구매건수,60대브랜드_구매비율
0,0,0.0,0.0
1,2,0.0,0.0
2,3,0.0,0.0
3,4,0.0,0.0
4,5,0.0,0.0
...,...,...,...
35962,49988,0.0,0.0
35963,49990,0.0,0.0
35964,49992,0.0,0.0
35965,49993,0.0,0.0


In [145]:
# 아동관련 상품 구매건수

f = data[data.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
f['baby_sales_rate'] = f['baby_sales'] / data.groupby('custid')['tot_amt'].count()
f = f.fillna(0)
features.append(f)
display(f)

,custid,baby_sales,baby_sales_rate
0,0,0,0.000000
1,2,1,0.000000
2,3,1,0.090909
3,4,0,0.000000
4,5,10,2.500000
...,...,...,...
35962,49988,2,0.200000
35963,49990,0,0.000000
35964,49992,1,0.166667
35965,49993,1,0.333333


In [146]:
# 식품 실구매액

food_list=['조리식품','축산물','농산물','수산물']
temp_f = data[data.net_amt>0].query("pc_nm == @food_list").groupby('custid')['net_amt'].agg([('food_sales', np.sum)])
f = pd.merge(unique_custid, temp_f, on = 'custid', how = 'left').fillna(0)
f['food_sales_rate'] = f['food_sales'] / data.groupby('custid')['net_amt'].sum()
f = f.fillna(0)

features.append(f)
display(f)

,custid,food_sales,food_sales_rate
0,0,0.0,0.000000
1,2,0.0,0.000000
2,3,25800.0,0.007380
3,4,0.0,0.000000
4,5,281944.0,0.275551
...,...,...,...
35962,49988,0.0,0.000000
35963,49990,0.0,0.000000
35964,49992,0.0,0.000000
35965,49993,0.0,0.000000


In [147]:
# 화장품관련 실구매액            

st = f"(net_amt > 0) and (pc_nm == '화장품')"
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_sales', np.sum)])
f= pd.merge(custid,t2, on = 'custid', how='left').fillna(0)
f['face_sales_rate'] = f['face_goods_sales'] / data.groupby('custid')['net_amt'].sum()
f = f.fillna(0)
features.append(f); display(f)

,custid,face_goods_sales,face_sales_rate
0,0,950400.0,0.606200
1,2,131400.0,0.000000
2,3,177650.0,0.050819
3,4,57000.0,0.025805
4,5,470700.0,0.460027
...,...,...,...
35962,49988,93100.0,0.124640
35963,49990,202350.0,0.000000
35964,49992,50350.0,0.213719
35965,49993,66500.0,0.375176


In [148]:
t2 = data.query(st).groupby('custid')['구매건수'].agg([('face_goods_sales_count', np.sum)])
f= pd.merge(custid,t2, on = 'custid', how='left').fillna(0)
f['face_sales_count_rate'] = f['face_goods_sales_count'] / data.groupby('custid')['구매건수'].sum()

f = f.fillna(0)
features.append(f); display(f)

,custid,face_goods_sales_count,face_sales_count_rate
0,0,8.0,0.727273
1,2,1.0,0.000000
2,3,1.0,0.111111
3,4,1.0,0.035714
4,5,4.0,1.000000
...,...,...,...
35962,49988,1.0,0.100000
35963,49990,1.0,0.000000
35964,49992,1.0,0.200000
35965,49993,1.0,0.333333


In [149]:
# 골프용품 구매비율

f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('golf_sales', lambda x: list(x).count('골프'))]).reset_index()
f['골프용품_구매비율'] = f['golf_sales'] / data.groupby('custid')['구매건수'].sum()
f = f.fillna(0)

features.append(f)
display(f)

,custid,golf_sales,골프용품_구매비율
0,0,0,0.000000
1,2,0,0.000000
2,3,4,0.444444
3,4,0,0.000000
4,5,3,0.750000
...,...,...,...
35962,49988,0,0.000000
35963,49990,0,0.000000
35964,49992,0,0.000000
35965,49993,0,0.000000


In [150]:
# 트랜드 명품 구매비율

sales_freq=data.query('tot_amt>0').groupby('custid')['sales_day'].count().reset_index()

f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('expensive_freq', lambda x: list(x).count('명품토탈')+list(x).count('명품시즌'))]).reset_index()
f['expensive_freq_rate'] = f['expensive_freq'] / data.groupby('custid')['tot_amt'].count()
f = f.fillna(0)

features.append(f)
display(f)

,custid,expensive_freq,expensive_freq_rate
0,0,1,0.090909
1,2,0,0.000000
2,3,0,0.000000
3,4,0,0.000000
4,5,0,0.000000
...,...,...,...
35962,49988,0,0.000000
35963,49990,0,0.000000
35964,49992,0,0.000000
35965,49993,0,0.000000


In [151]:
# 등급별로 나누어 계산

def fnc(m) :
    if m < 14720.0:
        return 'e등급'
    elif m < 31530.0 :
        return 'd등급'
    elif m < 57000.0 :
        return 'c등급'
    elif m < 104500.0:
        return 'b등급'
    else:
        return 'a등급' 
tmp_data = data[data['net_amt'] > 0]['net_amt'].apply(fnc).rename('구매등급')
data = pd.concat([data, tmp_data], axis = 1).fillna('환불')

In [152]:
f = pd.pivot_table(data = data, 
              columns = '구매등급',
              index = 'custid',
              values = '구매건수',
              aggfunc = 'sum',
              fill_value = 0).reset_index()
f['a등급구매비율'] = (f.a등급 / data.groupby('custid')['구매건수'].sum())
f['b등급구매비율'] = (f.b등급 / data.groupby('custid')['구매건수'].sum())
f['c등급구매비율'] = (f.c등급 / data.groupby('custid')['구매건수'].sum())
f['d등급구매비율'] = (f.d등급 / data.groupby('custid')['구매건수'].sum())
f['e등급구매비율'] = (f.e등급 / data.groupby('custid')['구매건수'].sum())
del f['환불']

f = f.fillna(0)
features.append(f)
display(f)

구매등급,custid,a등급,b등급,c등급,d등급,e등급,a등급구매비율,b등급구매비율,c등급구매비율,d등급구매비율,e등급구매비율
0,0,5,5,1,0,0,0.454545,0.454545,0.090909,0.000000,0.000000
1,2,9,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,7,10,5,4,2,0.777778,1.111111,0.555556,0.444444,0.222222
3,4,2,1,1,0,0,0.071429,0.035714,0.035714,0.000000,0.000000
4,5,14,10,5,3,0,3.500000,2.500000,1.250000,0.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,1,1,2,0,0,0.100000,0.100000,0.200000,0.000000,0.000000
35963,49990,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
35964,49992,1,0,1,0,0,0.200000,0.000000,0.200000,0.000000,0.000000
35965,49993,0,1,1,2,0,0.000000,0.333333,0.333333,0.666667,0.000000


In [167]:
data_2 = target.iloc[:len(target)]

In [153]:
# data_2 = pd.concat([id_data,target],axis = 1)
# data_2 = data_2.iloc[:len(target)]

In [168]:
len(target)

21587

In [175]:
data_2 = data_2.copy()

In [176]:
data_2['age'] = data_2['group'].str[1:]
data_2['age'] = data_2['age'].astype(int)

In [177]:
tr_data = pd.merge(tr,data_2,on="custid")

In [179]:
tr_cor_mean = dict(tr_data.groupby("corner_nm")["age"].mean())
tr_brd_mean = dict(tr_data.groupby("brd_nm")["age"].mean())
tr_pc_mean = dict(tr_data.groupby("pc_nm")["age"].mean())
tr_day_mean = dict(tr_data.groupby("sales_dayofweek")["age"].mean())

In [180]:
brd_data = tr.groupby("custid")["pc_nm"].agg([("pc","first")])

In [181]:
f = brd_data["pc"].apply(lambda x: tr_pc_mean[x])
features.append(f);display(f)

custid
0        31.040026
2        32.528038
3        31.989616
4        31.040026
5        34.171009
           ...    
49988    31.661859
49990    31.040026
49992    31.040026
49993    34.418536
49994    32.538421
Name: pc, Length: 35967, dtype: float64

In [182]:
d = tr.groupby("custid")["corner_nm"].agg([("cor","first")])
f = d["cor"].apply(lambda x: tr_cor_mean[x])
features.append(f);display(f)

custid
0        31.098543
2        34.180405
3        34.624397
4        31.098543
5        34.180405
           ...    
49988    31.453393
49990    31.098543
49992    31.098543
49993    32.817513
49994    32.525869
Name: cor, Length: 35967, dtype: float64

In [183]:
d = tr.groupby("custid")["sales_dayofweek"].agg([("day","first")])
f = d["day"].apply(lambda x: tr_day_mean[x])
features.append(f);display(f)

custid
0        32.647704
2        32.647704
3        31.988420
4        33.252828
5        33.252828
           ...    
49988    33.255179
49990    33.411906
49992    32.647704
49993    33.411906
49994    31.988420
Name: day, Length: 35967, dtype: float64

In [184]:
#주말방문비율
day = {'월' : 0 , '화' : 1 , '수' : 2 , '목' : 3 , '금' : 4 , '토': 5 , '일' : 6 }

tr['sales_dayofweek_num'] = tr['sales_dayofweek'].apply(lambda x : day[x] )

f = tr.groupby('custid')['sales_dayofweek_num'].agg([
    ('주말방문비율', lambda x: np.mean(x >4))]).reset_index()

features.append(f);display(f)

,custid,주말방문비율
0,0,0.636364
1,2,0.363636
2,3,0.366667
3,4,0.250000
4,5,0.156250
...,...,...
35962,49988,0.000000
35963,49990,0.000000
35964,49992,0.500000
35965,49993,0.000000


In [185]:
#평균할부개월수
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f);display(f)

,custid,평균할부개월수
0,0,2.8
1,2,3.5
2,3,2.7
3,4,4.5
4,5,1.9
...,...,...
35962,49988,2.2
35963,49990,6.0
35964,49992,1.0
35965,49993,1.0


In [186]:
X_train = pd.DataFrame({'custid': train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid')
display(X_train.shape)

X_test = pd.DataFrame({'custid': test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid')
display(X_test.shape)

C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'12_x', '11_x', '10_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_train = pd.merge(X_train, f, how='left',on='custid')
C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'9_x', '1_x', '8_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_train = pd.merge(X_train, f, how='left',on='custid')
C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'13_x', '14_x', '18_x', '11_x', '5_x', '7_x', '22_x', '16_x', '17_x', '10_x', '2_x', '4_x', '20_x', '19_x', '3_x', '15_x', '12_x', '21_x', '6_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_train = pd.merge(X_train, f, how='left',on='custid')
C:\Users\ch

(21587, 1806)

C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'12_x', '11_x', '10_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_test = pd.merge(X_test, f, how='left',on='custid')
C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'9_x', '1_x', '8_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_test = pd.merge(X_test, f, how='left',on='custid')
C:\Users\choij\AppData\Local\Temp\ipykernel_16976\1071587700.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'13_x', '14_x', '18_x', '11_x', '5_x', '7_x', '22_x', '16_x', '17_x', '10_x', '2_x', '4_x', '20_x', '19_x', '3_x', '15_x', '12_x', '21_x', '6_x'} in the result is deprecated and will raise a MergeError in a future version.
  X_test = pd.merge(X_test, f, how='left',on='custid')
C:\Users\choij\Ap

(14380, 1806)

In [187]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train["주말방문비율"])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균총구매액'] / X_train['평균할부개월수']
# X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

In [188]:
X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균총구매액'] / X_test['평균할부개월수']
# X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

In [165]:
X_train

,custid,refund_count,refund_bool,amt_refund,amt_refund_mean,amt_refund_max,love_brd_count,0,1_x,8_x,...,평균할부개월수,평균내점구매액,주중방문비율,국내상품_구매비율,할부구매가격,주말방문수,주중방문수,내점당편균구매건수,주중구매액,주말구매액
0,0,0,0,0,0.000000,0,4,0,0,0,...,2.8,174200.000000,99.363636,36.4,56558.441558,0,10,1.100000,1.730915e+06,11085.454545
1,2,1850500,1,-1807050,-164277.272727,-1416000,3,0,0,0,...,3.5,416761.111111,99.636364,90.9,97424.675325,0,9,1.222222,3.737211e+06,13639.454545
2,3,695600,1,-660820,-22027.333333,-589950,3,0,0,0,...,2.7,92020.000000,99.633333,100.0,28401.234568,0,25,1.200000,2.292065e+06,8435.166667
3,4,0,0,0,0.000000,0,1,0,0,0,...,4.5,261250.000000,99.750000,75.0,58055.555556,0,4,1.000000,1.042388e+06,2612.500000
4,5,0,0,0,0.000000,0,5,0,0,0,...,1.9,187176.259259,99.843750,81.2,83121.036184,0,27,1.185185,5.045863e+06,7896.498438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,1349000,1,-1281550,-16862.500000,-1145700,6,0,0,0,...,1.8,297069.272727,99.657895,89.5,119435.745614,0,55,1.381818,1.628291e+07,55895.928947
21583,29996,556000,1,-516000,-27157.894737,-184500,7,0,0,0,...,1.2,117275.000000,99.842105,89.5,61723.684211,0,12,1.583333,1.405078e+06,2222.052632
21584,29997,0,0,0,0.000000,0,8,0,0,0,...,1.5,203542.687500,99.200000,95.0,108556.100000,0,16,1.250000,3.230630e+06,26053.464000
21585,29998,55000,1,-52250,-4019.230769,-52250,2,0,0,0,...,2.2,56575.000000,99.461538,100.0,23737.762238,0,12,1.083333,6.752444e+05,3655.615385


In [166]:
X_test

,custid,refund_count,refund_bool,amt_refund,amt_refund_mean,amt_refund_max,love_brd_count,0,1_x,8_x,...,평균할부개월수,평균내점구매액,주중방문비율,국내상품_구매비율,할부구매가격,주말방문수,주중방문수,내점당편균구매건수,주중구매액,주말구매액
0,30001,64700,1,-61460,-2276.296296,-61460,4,0,0,0,...,1.7,231017.777778,99.259259,92.6,90595.206972,0,18,1.500000,4.127518e+06,30802.370370
1,30002,1018600,1,-973950,-9548.529412,-389500,14,0,0,0,...,2.1,122276.606061,99.705882,75.5,37676.265173,0,66,1.545455,8.046520e+06,23736.047059
2,30003,199900,1,-196800,-4187.234043,-138000,6,0,0,0,...,1.3,49772.142857,99.787234,89.4,22808.837971,0,28,1.678571,1.390655e+06,2965.148936
3,30005,0,0,0,0.000000,0,1,0,0,0,...,3.0,57000.000000,100.000000,0.0,19000.000000,0,1,1.000000,5.700000e+04,0.000000
4,30007,0,0,0,0.000000,0,1,0,0,0,...,1.3,49250.000000,99.333333,83.3,37884.615385,0,6,1.000000,2.935300e+05,1970.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,49988,0,0,0,0.000000,0,1,0,0,0,...,2.2,158000.000000,100.000000,75.0,71818.181818,0,4,1.000000,6.320000e+05,0.000000
14376,49990,0,0,0,0.000000,0,1,0,0,0,...,6.0,213000.000000,100.000000,0.0,35500.000000,0,1,1.000000,2.130000e+05,0.000000
14377,49992,0,0,0,0.000000,0,1,0,0,0,...,1.0,110500.000000,99.500000,50.0,110500.000000,0,2,1.000000,2.198950e+05,1105.000000
14378,49993,0,0,0,0.000000,0,2,0,0,0,...,1.0,35757.250000,100.000000,100.0,35757.250000,0,4,1.000000,1.430290e+05,0.000000


In [164]:
pd.DataFrame(X_train).to_csv("num_features_train.csv",index = False,encoding = 'utf-8')
pd.DataFrame(X_test).to_csv("num_features_test.csv",index = False,encoding = 'utf-8')

***